In [91]:
import numpy as np
import pandas as pd
import re
from thefuzz import process

In [92]:
df = pd.read_csv('/home/j/Documents/Projects/social-media-combat-detection/data/annotated.csv')

In [93]:
df.head()

,idx,msg_txt,annotation
0,0,"In Kyiv, fragments of a downed rocket damage...",1
1,1,"Kyiv region is attacked by drones again, air...",1
2,2,Explosions are also heard in the capital. Ai...,1
3,4,Kyiv region is attacked by drones - Kuleba ...,1
4,5,The rocket that fell in Shevchenkivskyi dist...,0


In [94]:
def extract_capitalized_words(input_string):
        # Define a regular expression pattern to match capitalized words
        pattern = r'\b[A-Z][a-z]*\b'
        
        # Use re.findall to find all matches in the input string
        capitalized_words = re.findall(pattern, input_string)
        
        return capitalized_words

In [7]:
coord_dict = {'Біле': [48.4936, 39.0367], 'Bile': [48.4936, 39.0367], 'Белое': [48.4936, 39.0367], 'Снятин': [48.45, 25.5667], 'Sniatyn': [48.45, 25.5667], 'Snyatyn': [48.45, 25.5667], 'Снятын': [48.45, 25.5667], 'Śniatyn': [48.45, 25.5667], 'Sneatîn': [48.45, 25.5667], 'Великий Дальник': [46.4658, 30.5583], 'Velykyi Dalnyk': [46.4658, 30.5583], 'Velikiy Dalnik': [46.4658, 30.5583], 'Сніжне': [48.0167, 38.7667], 'Snizhne': [48.0167, 38.7667], 'Snezhnoye': [48.0167, 38.7667], 'Снежное': [48.0167, 38.7667], 'Василівка': [47.4344, 35.2808], 'Васильевка': [47.4344, 35.2808], 'Vasylivka': [47.4344, 35.2808], 'Вінниця': [49.2372, 28.4672], 'Vinnytsia': [49.2372, 28.4672], 'Vinnytsya': [49.2372, 28.4672], 'Vinnitsa': [49.2372, 28.4672], 'Vinitsia': [49.2372, 28.4672], 'Винница': [49.2372, 28.4672], 'Коростень': [50.95, 28.65], 'Korosten': [50.95, 28.65], 'Iskorosten': [50.95, 28.65], 'Іскоростень': [50.95, 28.65], 'Брацлав': [48.8147, 28.9447], 'Bratslav': [48.8147, 28.9447], 'Braclav': [48.8147, 28.9447], 'Bracław': [48.8147, 28.9447], 'Рогатин': [49.4167, 24.6167], 'Rohatyn': [49.4167, 24.6167], 'Rogatyn': [49.4167, 24.6167], 'Rogatin': [49.4167, 24.6167], 'Долина': [48.9703, 24.0108], 'Dolyna': [48.9703, 24.0108], 'Dolina': [48.9703, 24.0108], 'Володимир-Волинський': [50.8481, 24.3222], 'Volodymyr': [50.8481, 24.3222], 'Vladimir': [50.8481, 24.3222], 'Владимир': [50.8481, 24.3222], 'Владимир-Волынский': [50.8481, 24.3222], 'Volodymyr-Volynskyi': [50.8481, 24.3222], 'Vladimir-Volynskiy': [50.8481, 24.3222], 'Перечин': [48.7333, 22.4667], 'Perechyn': [48.7333, 22.4667], 'Perechin': [48.7333, 22.4667], 'Сквира': [49.7333, 29.6833], 'Skvyra': [49.7333, 29.6833], 'Skvira': [49.7333, 29.6833], 'Яворів': [49.9469, 23.3931], 'Yavoriv': [49.9469, 23.3931], 'Yavorov': [49.9469, 23.3931], 'Яворов': [49.9469, 23.3931], 'Jaworów': [49.9469, 23.3931], 'Торецьк': [48.4, 37.8333], 'Toretsk': [48.4, 37.8333], 'Торецк': [48.4, 37.8333], 'Дзержинськ': [50.1517, 27.9392], 'Дзержинск': [48.4, 37.8333], 'Dzerzhynsk': [50.1517, 27.9392], 'Dzerzhinsk': [48.4, 37.8333], 'Лозова': [48.8892, 36.3161], 'Lozova': [48.8892, 36.3161], 'Lozovaya': [48.8892, 36.3161], 'Лозовая': [48.8892, 36.3161], 'Чернігівка': [47.1942, 36.2023], 'Chernihivka': [47.1942, 36.2023], 'Chernigovka': [47.1942, 36.2023], 'Черниговка': [47.1942, 36.2023], 'Tschernihivka': [47.1942, 36.2023], 'Якимівка': [46.6967, 35.1664], 'Yakymivka': [46.6967, 35.1664], 'Yakimovka': [46.6967, 35.1664], 'Akimovka': [46.6967, 35.1664], 'Акимовка': [46.6967, 35.1664], 'Трускавець': [49.2806, 23.505], 'Truskavets': [49.2806, 23.505], 'Трускавец': [49.2806, 23.505], 'Truskawiec': [49.2806, 23.505], 'Фастів': [50.0747, 29.9181], 'Fastiv': [50.0747, 29.9181], 'Fastov': [50.0747, 29.9181], 'Фастов': [50.0747, 29.9181], 'Винники': [49.8156, 24.1297], 'Vynnyky': [49.8156, 24.1297], 'Vinniki': [49.8156, 24.1297], 'Вахрушеве': [48.1606, 38.8389], 'Vakhrusheve': [48.1606, 38.8389], 'Vakhrushevo': [48.1606, 38.8389], 'Вахрушево': [48.1606, 38.8389], 'Гурзуф': [44.5528, 34.2875], 'Hurzuf': [44.5528, 34.2875], 'Gurzuf': [44.5528, 34.2875], 'Γορζουβίται': [44.5528, 34.2875], 'Губиниха': [48.8084, 35.2535], 'Hubynykha': [48.8084, 35.2535], 'Gubiniha': [48.8084, 35.2535], 'Івано-Франківськ': [48.9228, 24.7106], 'Ivano-Frankivsk': [48.9228, 24.7106], 'Ivano-Frankovsk': [48.9228, 24.7106], 'Ивано-Франковск': [48.9228, 24.7106], 'Iwano-Frankivsk': [48.9228, 24.7106], 'Ужгород': [48.6239, 22.295], 'Uzhhorod': [48.6239, 22.295], 'Uzhgorod': [48.6239, 22.295], 'Любомль': [51.2158, 24.0408], 'Liuboml': [51.2158, 24.0408], 'Lyuboml': [51.2158, 24.0408], 'Luboml': [51.2158, 24.0408], 'Соснівка': [50.2946, 24.2525], 'Sosnivka': [50.2946, 24.2525], 'Sosnovka': [50.2946, 24.2525], 'Сосновка': [50.2946, 24.2525], 'Sosnówka': [50.2946, 24.2525], 'Підволочиськ': [49.5311, 26.1467], 'Pidvolochysk': [49.5311, 26.1467], 'Podvolochisk': [49.5311, 26.1467], 'Подволочиск': [49.5311, 26.1467], 'Podwołoczyska': [49.5311, 26.1467], 'Vasilivka': [47.4344, 35.2808], 'Vasilyevka': [47.4344, 35.2808], 'Vasilevka': [47.4344, 35.2808], 'Новоселиця': [48.2167, 26.2667], 'Novoselytsia': [48.2167, 26.2667], 'Novoselitsa': [48.2167, 26.2667], 'Новоселица': [48.2167, 26.2667], 'Великі Лучки': [48.42, 22.5611], 'Velyki Luchky': [48.42, 22.5611], 'Velikie Luchki': [48.42, 22.5611], 'Великие Лучки': [48.42, 22.5611], 'Velyki Lučky': [48.42, 22.5611], 'Nagylucska': [48.42, 22.5611], 'Veľké Lúčky': [48.42, 22.5611], 'Веселинове': [47.3583, 31.2397], 'Veselynove': [47.3583, 31.2397], 'Veselinovo': [47.3583, 31.2397], 'Веселиново': [47.3583, 31.2397], 'Чигирин': [49.0833, 32.6667], 'Chyhyryn': [49.0833, 32.6667], 'Chihyrin': [49.0833, 32.6667], 'Заводське': [50.4002, 33.3908], 'Zavodske': [50.4002, 33.3908], 'Zavodskoe': [50.4002, 33.3908], 'Zavodskoye': [50.4002, 33.3908], 'Заводское': [50.4002, 33.3908], 'Покровське': [47.9853, 36.2367], 'Pokrovske': [47.9853, 36.2367], 'Pokrovskoe': [47.9853, 36.2367], 'Покровское': [47.9853, 36.2367], 'Дніпровське': [48.5953, 34.4207], 'Dniprovske': [48.5953, 34.4207], 'Dnieprovskoe': [48.5953, 34.4207], 'Dneprovskoye': [48.5953, 34.4207], 'Днепровское': [48.5953, 34.4207], 'Солоне': [48.2054, 34.8686], 'Solone': [48.2054, 34.8686], 'Solonoe': [48.2054, 34.8686], 'Solyonoe': [48.2054, 34.8686], 'Солёное': [48.2054, 34.8686], 'Савинці': [49.4105, 37.0617], 'Savyntsi': [49.4105, 37.0617], 'Savintsy': [49.4105, 37.0617], 'Савинцы': [49.4105, 37.0617], 'Царичанка': [48.9432, 34.4748], 'Tsarychanka': [48.9432, 34.4748], 'Tsarichanka': [48.9432, 34.4748], 'Коростишів': [50.3186, 29.0592], 'Korostyshiv': [50.3186, 29.0592], 'Korostyshov': [50.3186, 29.0592], 'Korostishov': [50.3186, 29.0592], 'Коростышов': [50.3186, 29.0592], 'Korosteszów': [50.3186, 29.0592], 'Бахмут': [48.6044, 38.0067], 'Bakhmut': [48.6044, 38.0067], 'Bahmut': [48.6044, 38.0067], 'Артемівськ': [48.4403, 38.7422], 'Артёмовск': [48.4403, 38.7422], 'Artemivsk': [48.4403, 38.7422], 'Artemovsk': [48.4403, 38.7422], 'Artyomovsk': [48.4403, 38.7422], 'Томашпіль': [48.5471, 28.5157], 'Tomashpil': [48.5471, 28.5157], 'Tomashpol': [48.5471, 28.5157], 'Томашполь': [48.5471, 28.5157], 'Tomaszpol': [48.5471, 28.5157], 'Ізмаїл': [45.35, 28.8333], 'Izmail': [45.35, 28.8333], 'Izmayil': [45.35, 28.8333], 'Измаил': [45.35, 28.8333], 'Исмаил': [45.35, 28.8333], 'Ismail': [45.35, 28.8333], 'Smil': [45.35, 28.8333], 'Izmaił': [45.35, 28.8333], 'Барвінкове': [48.9067, 37.0131], 'Barvinkove': [48.9067, 37.0131], 'Barvenkove': [48.9067, 37.0131], 'Barvenkovo': [48.9067, 37.0131], 'Барвенково': [48.9067, 37.0131], 'Камінь-Каширський': [51.6242, 24.9606], 'Kamin-Kashyrskyi': [51.6242, 24.9606], 'Kamen-Kashyrskyi': [51.6242, 24.9606], 'Камень-Каширский': [51.6242, 24.9606], 'Kamień Koszyrski': [51.6242, 24.9606], 'Борислав': [49.2881, 23.4267], 'Boryslav': [49.2881, 23.4267], 'Borislav': [49.2881, 23.4267], 'Borysław': [49.2881, 23.4267], 'Гайсин': [48.8094, 29.3906], 'Haisyn': [48.8094, 29.3906], 'Haysyn': [48.8094, 29.3906], 'Hajsyn': [48.8094, 29.3906], 'Лиман': [48.9861, 37.8111], 'Lyman': [48.9861, 37.8111], 'Liman': [48.9861, 37.8111], 'Лыман': [48.9861, 37.8111], 'Красний Лиман': [48.9861, 37.8111], 'Красный Лыман': [48.9861, 37.8111], 'Krasnyi Lyman': [48.9861, 37.8111], 'Рубіжне': [49.0167, 38.3667], 'Rubizhne': [49.0167, 38.3667], 'Rubezhne': [49.0167, 38.3667], 'Rubezhnoe': [49.0167, 38.3667], 'Рубежное': [49.0167, 38.3667], 'Бородянка': [50.6439, 29.9339], 'Borodianka': [50.6439, 29.9339], 'Borodyanka': [50.6439, 29.9339], 'Броварський': [50.5114, 30.7903], 'Brovary': [50.5114, 30.7903], 'Brovari': [50.5114, 30.7903], 'Бровары': [50.5114, 30.7903], 'Browary': [50.5114, 30.7903], 'Дніпрорудне': [47.3855, 34.9879], 'Dniprorudne': [47.3855, 34.9879], 'Dnieprorudne': [47.3855, 34.9879], 'Dneprorudnoye': [47.3855, 34.9879], 'Dnieprorudnoye': [47.3855, 34.9879], 'Днепрорудное': [47.3855, 34.9879], 'Піщанка': [48.2058, 28.8889], 'Pishchanka': [48.2058, 28.8889], 'Peschanka': [48.2058, 28.8889], 'Песчанка': [48.2058, 28.8889], 'Piszczanka': [48.2058, 28.8889], 'Зміїв': [49.6878, 36.3558], 'Zmiiv': [49.6878, 36.3558], 'Zmiev': [49.6878, 36.3558], 'Zmiyev': [49.6878, 36.3558], 'Змиёв': [49.6878, 36.3558], 'Верхньодніпровськ': [48.6561, 34.3283], 'Verkhnodniprovsk': [48.6561, 34.3283], 'Verkhnedneprovsk': [48.6561, 34.3283], 'Verkhnednieprovsk': [48.6561, 34.3283], 'Verkhnednyeprovsk': [48.6561, 34.3283], 'Верхнеднепровск': [48.6561, 34.3283], 'Амвросіївка': [47.7958, 38.4801], 'Amvrosiivka': [47.7958, 38.4801], 'Amvrosiyivka': [47.7958, 38.4801], 'Амворосиевка': [47.7958, 38.4801], 'Amvrosivka': [47.7958, 38.4801], 'Суходільськ': [48.35, 39.7167], 'Sukhodilsk': [48.35, 39.7167], 'Sukhodolsk': [48.35, 39.7167], 'Суходольск': [48.35, 39.7167], 'Кегичівка': [49.2858, 35.7622], 'Kehychivka': [49.2858, 35.7622], 'Kegichevka': [49.2858, 35.7622], 'Кегичёвка': [49.2858, 35.7622], 'Софіївка': [48.2683, 38.1847], 'Sofiivka': [48.2683, 38.1847], 'Sofiyivka': [48.2683, 38.1847], 'Sofievka': [48.2683, 38.1847], 'Sofiyevka': [48.2683, 38.1847], 'Софиевка': [48.2683, 38.1847], 'Ясіня': [48.2728, 24.3747], 'Yasinia': [48.2728, 24.3747], 'Yasinya': [48.2728, 24.3747], 'Ясиня': [48.2728, 24.3747], 'Körösmező': [48.2728, 24.3747], 'Jasiňa': [48.2728, 24.3747], 'Нова Маячка': [46.6033, 33.2283], 'Nova Maiachka': [46.6033, 33.2283], 'Nova Mayachka': [46.6033, 33.2283], 'Novaya Mayachka': [46.6033, 33.2283], 'Новая Маячка': [46.6033, 33.2283], 'Авдіївка': [48.1333, 37.75], 'Avdiivka': [48.1333, 37.75], 'Avdeevka': [48.1333, 37.75], 'Авдеевка': [48.1333, 37.75], 'Вапнярка': [48.5333, 28.75], 'Vapniarka': [48.5333, 28.75], 'Vapnyarka': [48.5333, 28.75], 'Vapniarca': [48.5333, 28.75], 'Wapnjarka': [48.5333, 28.75], 'Wapniarka': [48.5333, 28.75], 'Диканька': [49.8222, 34.5341], 'Dykanka': [49.8222, 34.5341], 'Dikanka': [49.8222, 34.5341], 'Драбів': [49.9603, 32.1481], 'Drabiv': [49.9603, 32.1481], 'Drabov': [49.9603, 32.1481], 'Драбов': [49.9603, 32.1481], 'Маневичі': [51.2925, 25.5503], 'Manevychi': [51.2925, 25.5503], 'Маневичи': [51.2925, 25.5503], 'Maniewicze': [51.2925, 25.5503], 'Manewytschi': [51.2925, 25.5503], 'Алушта': [44.6672, 34.3978], 'Alushta': [44.6672, 34.3978], 'Alusta': [44.6672, 34.3978], 'Нетішин': [50.33, 26.64], 'Netishyn': [50.33, 26.64], 'Netishin': [50.33, 26.64], 'Neteshin': [50.33, 26.64], 'Нетешин': [50.33, 26.64], 'Niecieszyn': [50.33, 26.64], 'Берегове': [48.2025, 22.6375], 'Berehove': [48.2025, 22.6375], 'Beregove': [48.2025, 22.6375], 'Beregovo': [48.2025, 22.6375], 'Beregovoe': [48.2025, 22.6375], 'Береговое': [48.2025, 22.6375], 'Beregszász': [48.2025, 22.6375], 'Кропивницький': [48.5103, 32.2667], 'Kropyvnytskyi': [48.5103, 32.2667], 'Kropivnitskyi': [48.5103, 32.2667], 'Kropivnitskiy': [48.5103, 32.2667], 'Кропивницкий': [48.5103, 32.2667], 'Kirovohrad': [48.5103, 32.2667], 'Kirovograd': [48.5103, 32.2667], 'Софіївська Борщагівка': [50.4114, 30.3692], 'Sofiiivska Borshchahivka': [50.4114, 30.3692], 'Sofievskaya Borschagovka': [50.4114, 30.3692], 'Софиевская Борщаговка': [50.4114, 30.3692], 'Ладижин': [48.6667, 29.2167], 'Ladyzhyn': [48.6667, 29.2167], 'Ladyzhin': [48.6667, 29.2167], 'Ладыжин': [48.6667, 29.2167], 'Ładyżyn': [48.6667, 29.2167], 'Єнакієве': [48.2311, 38.2053], 'Yenakiieve': [48.2311, 38.2053], 'Yenakiyevo': [48.2311, 38.2053], 'Енакиево': [48.2311, 38.2053], 'Валки': [49.8386, 35.6217], 'Valky': [49.8386, 35.6217], 'Valki': [49.8386, 35.6217], 'Вільнянськ': [47.9445, 35.4331], 'Vilniansk': [47.9445, 35.4331], 'Vilnyansk': [47.9445, 35.4331], 'Volnyansk': [47.9445, 35.4331], 'Вольнянск': [47.9445, 35.4331], 'Старий Самбір': [49.4333, 23], 'Staryi Sambir': [49.4333, 23], 'Staryi Sambor': [49.4333, 23], 'Старый Самбор': [49.4333, 23], 'Stary Sambor': [49.4333, 23], 'Staremiasto': [49.4333, 23], 'Stare Miasto': [49.4333, 23], 'Пулини': [50.4664, 28.2778], 'Pulyny': [50.4664, 28.2778], 'Puliny': [50.4664, 28.2778], 'Пулины': [50.4664, 28.2778], 'Червоноармійськ': [50.4664, 28.2778], 'Червоноармейск': [50.4664, 28.2778], 'Chervonoarmiisk': [50.4664, 28.2778], 'Великодолинське': [46.3425, 30.5653], 'Velykodolynske': [46.3425, 30.5653], 'Velikodolinskoe': [46.3425, 30.5653], 'Великодолинское': [46.3425, 30.5653], 'Приморськ': [46.7333, 36.3596], 'Prymorsk': [46.7333, 36.3596], 'Primorsk': [46.7333, 36.3596], 'Приморск': [46.7333, 36.3596], 'Баранівка': [50.3, 27.6667], 'Baranivka': [50.3, 27.6667], 'Baranovka': [50.3, 27.6667], 'Барановка': [50.3, 27.6667], 'Сватове': [49.4092, 38.1619], 'Svatove': [49.4092, 38.1619], 'Svatovo': [49.4092, 38.1619], 'Сватово': [49.4092, 38.1619], 'Путивль': [51.3347, 33.8686], 'Putyvl': [51.3347, 33.8686], 'Putivl': [51.3347, 33.8686], 'Вишкове': [48.0581, 23.4164], 'Vyshkove': [48.0581, 23.4164], 'Vyshkovo': [48.0581, 23.4164], 'Вышково': [48.0581, 23.4164], 'Visk': [48.0581, 23.4164], 'Тульчин': [48.6744, 28.8497], 'Tulchyn': [48.6744, 28.8497], 'Tulchin': [48.6744, 28.8497], 'Tulczyn': [48.6744, 28.8497], 'Tulcin': [48.6744, 28.8497], 'Nesterwar': [48.6744, 28.8497], 'Хуст': [48.1814, 23.2978], 'Khust': [48.1814, 23.2978], 'Hust': [48.1814, 23.2978], 'Chust': [48.1814, 23.2978], 'Huszt': [48.1814, 23.2978], "Кам'янське": [48.5076, 34.6132], 'Kamianske': [48.5076, 34.6132], 'Kamyanske': [48.5076, 34.6132], 'Каменское': [48.5076, 34.6132], 'Днепродзержинск': [48.5076, 34.6132], 'Dniprodzerzhynsk': [48.5076, 34.6132], 'Dneprodzerzhinsk': [48.5076, 34.6132], 'Хмільник': [49.55, 27.9667], 'Khmilnyk': [49.55, 27.9667], 'Khmelnik': [49.55, 27.9667], 'Хмельник': [49.55, 27.9667], 'Chmielnik': [49.55, 27.9667], 'Чинаді́йово': [48.4833, 22.8333], 'Chynadiiovo': [48.4833, 22.8333], 'Chynadiyovo': [48.4833, 22.8333], 'Chynadiieve': [48.4833, 22.8333], 'Chinadievo': [48.4833, 22.8333], 'Чинадиево': [48.4833, 22.8333], 'Szentmiklós': [48.4833, 22.8333], 'Činadno': [48.4833, 22.8333], 'Більмак': [47.3589, 36.65], 'Bilmak': [47.3589, 36.65], 'Belmak': [47.3589, 36.65], 'Бильмак': [47.3589, 36.65], 'Куйбишеве': [47.3589, 36.65], 'Куйбышево': [47.3589, 36.65], 'Kuibysheve': [47.3589, 36.65], 'Kuibyshevo': [47.3589, 36.65], 'Білгород-Дністровський': [46.1833, 30.3333], 'Bilhorod-Dnistrovskyi': [46.1833, 30.3333], 'Bilhorod-Dnestrovsky': [46.1833, 30.3333], 'Belgorod-Dnestrovskiy': [46.1833, 30.3333], 'Белгород-Днестровский': [46.1833, 30.3333], "Кам'янка-Дніпровська": [47.4792, 34.4232], 'Kamianka-Dniprovska': [47.4792, 34.4232], 'Kamenka-Dneprovskaya': [47.4792, 34.4232], 'Каменка-Днепровская': [47.4792, 34.4232], 'Нова Борова': [50.6922, 28.6367], 'Nova Borova': [50.6922, 28.6367], 'Novaya Borovaya': [50.6922, 28.6367], 'Новая Боровая': [50.6922, 28.6367], 'Українка': [50.15, 30.7519], 'Ukrainka': [50.15, 30.7519], 'Ukrayinka': [50.15, 30.7519], 'Украинка': [50.15, 30.7519], 'Біленьке': [48.7664, 37.6314], 'Bilenke': [48.7664, 37.6314], 'Belenkoe': [48.7664, 37.6314], 'Беленькое': [48.7664, 37.6314], 'Чоп': [48.4333, 22.2], 'Chop': [48.4333, 22.2], 'Жовква': [50.0667, 23.9667], 'Zhovkva': [50.0667, 23.9667], 'Zholkva': [50.0667, 23.9667], 'Жолква': [50.0667, 23.9667], 'Żółkiew': [50.0667, 23.9667], 'Аграрне': [45.0186, 34.0544], 'Ahrarne': [45.0186, 34.0544], 'Agrarne': [45.0186, 34.0544], 'Agrarnoe': [45.0186, 34.0544], 'Аграрное': [45.0186, 34.0544], 'Тараща': [49.55, 30.5], 'Tarashcha': [49.55, 30.5], 'Tarascha': [49.55, 30.5], 'Добровеличківка': [48.35, 31.1667], 'Dobrovelychkivka': [48.35, 31.1667], 'Dobrovelichkovka': [48.35, 31.1667], 'Добровеличковка': [48.35, 31.1667], 'Петропавлівська Борщагівка': [50.4378, 30.3439], 'Petropavlivska Borshchahivka': [50.4378, 30.3439], 'Petropavlovskaya Borschagovka': [50.4378, 30.3439], 'Петропавловская Борщаговка': [50.4378, 30.3439], 'Сосниця': [51.5222, 32.5017], 'Sosnytsia': [51.5222, 32.5017], 'Sosnitsa': [51.5222, 32.5017], 'Сосница': [51.5222, 32.5017], 'Шахтарськ': [48.0333, 38.4833], 'Shakhtarsk': [48.0333, 38.4833], 'Shakhtyorsk': [48.0333, 38.4833], 'Шахтёрск': [48.0333, 38.4833], 'Шепетівка': [50.1833, 27.0667], 'Shepetivka': [50.1833, 27.0667], 'Shepetovka': [50.1833, 27.0667], 'Шепетовка': [50.1833, 27.0667], 'Szepetówka': [50.1833, 27.0667], 'Генічеськ': [46.1769, 34.7989], 'Henichesk': [46.1769, 34.7989], 'Genichesk': [46.1769, 34.7989], 'Геническ': [46.1769, 34.7989], 'Веселе': [47.016, 34.9124], 'Vesele': [47.016, 34.9124], 'Veseloye': [47.016, 34.9124], 'Veseloe': [47.016, 34.9124], 'Весёлое': [47.016, 34.9124], 'Галич': [49.1128, 24.7217], 'Halych': [49.1128, 24.7217], 'Galich': [49.1128, 24.7217], 'Halici': [49.1128, 24.7217], 'Halicz': [49.1128, 24.7217], 'Halytsch': [49.1128, 24.7217], 'Halitsch': [49.1128, 24.7217], 'Galitsch': [49.1128, 24.7217], 'Хрестівка': [48.15, 38.3667], 'Khrestivka': [48.15, 38.3667], 'Khrestovka': [48.15, 38.3667], 'Крестовка': [48.15, 38.3667], 'Кіровське': [48.15, 38.3667], 'Кировское': [48.15, 38.3667], 'Kirovske': [48.15, 38.3667], 'Kirovskoe': [48.15, 38.3667], 'Чугуїв': [49.8353, 36.6756], 'Chuhuiv': [49.8353, 36.6756], 'Chuguev': [49.8353, 36.6756], 'Чугуев': [49.8353, 36.6756], 'Біла Церква': [49.7956, 30.1167], 'Bila Tserkva': [49.7956, 30.1167], 'Bila Cerkva': [49.7956, 30.1167], 'Belaya Tserkva': [49.7956, 30.1167], 'Belaya Tserkov': [49.7956, 30.1167], 'Белая Церковь': [49.7956, 30.1167], 'Била Церква': [49.7956, 30.1167], 'Biała Cerkiew': [49.7956, 30.1167], 'Костянтинівка': [48.5333, 37.7167], 'Kostiantynivka': [48.5333, 37.7167], 'Kostyantynivka': [48.5333, 37.7167], 'Kostiantinovka': [48.5333, 37.7167], 'Kostyantinovka': [48.5333, 37.7167], 'Константиновка': [48.5333, 37.7167], 'Димер': [50.7872, 30.3117], 'Dymer': [50.7872, 30.3117], 'Dymier': [50.7872, 30.3117], 'Дымер': [50.7872, 30.3117], 'Велика Димерка': [50.5928, 30.9103], 'Velyka Dymerka': [50.5928, 30.9103], 'Velikaya Dymerka': [50.5928, 30.9103], 'Великая Дымерка': [50.5928, 30.9103], 'Дніпро': [48.45, 34.9833], 'Dnipro': [48.45, 34.9833], 'Dnepr': [48.45, 34.9833], 'Dnieper': [48.45, 34.9833], 'Dniepr': [48.45, 34.9833], 'Dniepropetrovsk': [48.45, 34.9833], 'Dnepropetrovsk': [48.45, 34.9833], 'Dnipropetrovsk': [48.45, 34.9833], 'Днепр': [48.45, 34.9833], 'Днепропетровск': [48.45, 34.9833], 'Чечельник': [48.2122, 29.3622], 'Chechelnyk': [48.2122, 29.3622], 'Chichelnik': [48.2122, 29.3622], 'Chetschelnik': [48.2122, 29.3622], 'Chitchilnik': [48.2122, 29.3622], 'Cicelnic': [48.2122, 29.3622], 'Czeczelnik': [48.2122, 29.3622], 'Tschetschelnik': [48.2122, 29.3622], 'Бурородчани': [48.8, 24.5333], 'Bohorodchany': [48.8, 24.5333], 'Bogorodchani': [48.8, 24.5333], 'Богородчаны': [48.8, 24.5333], 'Ніжин': [51.05, 31.9], 'Nizhyn': [51.05, 31.9], 'Nezhin': [51.05, 31.9], 'Nizhin': [51.05, 31.9], 'Нежин': [51.05, 31.9], 'Nieżyn': [51.05, 31.9], 'Новоград-Волинський': [50.5833, 27.6333], 'Novohrad-Volynskyi': [50.5833, 27.6333], 'Novograd-Volynsky': [50.5833, 27.6333], 'Novograd-Volynskiy': [50.5833, 27.6333], 'Новоград-Волынский': [50.5833, 27.6333], 'Zwiahel': [50.5833, 27.6333], 'Умань': [48.75, 30.2167], 'Uman': [48.75, 30.2167], 'Humań': [48.75, 30.2167], 'Олександрійське': [48.6019, 32.9889], 'Oleksandriiske': [48.6019, 32.9889], 'Alexandriyskoe': [48.6019, 32.9889], 'Александрийское': [48.6019, 32.9889], 'Димитрове': [48.6019, 32.9889], 'Димитрово': [48.6019, 32.9889], 'Dymytrove': [48.6019, 32.9889], 'Dimitrovo': [48.6019, 32.9889], 'Бердичів': [49.8919, 28.6], 'Berdychiv': [49.8919, 28.6], 'Berdichiv': [49.8919, 28.6], 'Бердычив': [49.8919, 28.6], 'Молодогвардійськ': [48.3444, 39.6583], 'Molodohvardiisk': [48.3444, 39.6583], 'Molodogvardeysk': [48.3444, 39.6583], 'Молодогвардейск': [48.3444, 39.6583], 'Городище': [49.2925, 31.4581], 'Horodyshche': [49.2925, 31.4581], 'Gorodyshche': [49.2925, 31.4581], 'Gorodishche': [49.2925, 31.4581], 'Іллінці': [49.1, 29.2], 'Illintsi': [49.1, 29.2], 'Ilyintsi': [49.1, 29.2], 'Ильинцы': [49.1, 29.2], 'Ilińce': [49.1, 29.2], 'Ставище': [49.3914, 30.1917], 'Stavyshche': [49.3914, 30.1917], 'Stavische': [49.3914, 30.1917], 'Кагарлик': [49.8522, 30.8092], 'Kaharlyk': [49.8522, 30.8092], 'Kagarlyk': [49.8522, 30.8092], 'Kagarlik': [49.8522, 30.8092], 'Кагарлык': [49.8522, 30.8092], 'Світловодськ': [49.0833, 33.25], 'Svitlovodsk': [49.0833, 33.25], 'Svetlovodsk': [49.0833, 33.25], 'Светловодск': [49.0833, 33.25], 'Семенівка': [52.1772, 32.5797], 'Semenivka': [52.1772, 32.5797], 'Semenovka': [52.1772, 32.5797], 'Семёновка': [52.1772, 32.5797], "Слов'янськ": [48.87, 37.625], 'Sloviansk': [48.87, 37.625], 'Slavyansk': [48.87, 37.625], 'Slovyansk': [48.87, 37.625], 'Славянск': [48.87, 37.625], 'Болград': [45.6855, 28.6134], 'Bolhrad': [45.6855, 28.6134], 'Bolgrad': [45.6855, 28.6134], 'Слобожанське': [48.5342, 35.0749], 'Slobozhanske': [48.5342, 35.0749], 'Slobozhanskoe': [48.5342, 35.0749], 'Slobozhanskoye': [48.5342, 35.0749], 'Слобожанское': [48.5342, 35.0749], 'Ювілейне': [48.5342, 35.0749], 'Ювилейное': [48.5342, 35.0749], 'Yuvileine': [48.5342, 35.0749], 'Ватутіне': [49.0167, 31.0667], 'Vatutine': [49.0167, 31.0667], 'Vatutino': [49.0167, 31.0667], 'Ватутино': [49.0167, 31.0667], 'Львів': [49.8419, 24.0315], 'Lviv': [49.8419, 24.0315], 'Lvov': [49.8419, 24.0315], 'Львов': [49.8419, 24.0315], 'Lwów': [49.8419, 24.0315], 'Lemberg': [49.8419, 24.0315], 'Lavov': [49.8419, 24.0315], 'Liov': [49.8419, 24.0315], 'Lvoŭ': [49.8419, 24.0315], 'Leopolis': [49.8419, 24.0315], 'Leo': [49.8419, 24.0315], 'Lwiw': [49.8419, 24.0315], 'İlbav': [49.8419, 24.0315], 'Ilyvó': [49.8419, 24.0315], 'Ilov': [49.8419, 24.0315], 'Ilôv': [49.8419, 24.0315], 'Мала Данилівка': [50.0614, 36.1675], 'Mala Danylivka': [50.0614, 36.1675], 'Mala Danilovka': [50.0614, 36.1675], 'Malaya Danilovka': [50.0614, 36.1675], 'Малая Даниловка': [50.0614, 36.1675], 'Запоріжжя': [47.8378, 35.1383], 'Zaporizhzhia': [47.8378, 35.1383], 'Zaporizhia': [47.8378, 35.1383], 'Zaporozhye': [47.8378, 35.1383], 'Zaporozhie': [47.8378, 35.1383], 'Запорожье': [47.8378, 35.1383], 'Кальміуське': [47.6667, 38.0667], 'Kalmiuske': [47.6667, 38.0667], 'Kalmiske': [47.6667, 38.0667], 'Кальмиусское': [47.6667, 38.0667], 'Комсомольское': [47.6667, 38.0667], 'Komsomolske': [47.6667, 38.0667], 'Komsomolskoye': [47.6667, 38.0667], 'Первомайський': [49.3869, 36.2142], 'Pervomaiskyi': [49.3869, 36.2142], 'Pervomaysky': [49.3869, 36.2142], 'Pervomayskyi': [49.3869, 36.2142], 'Likhachove': [49.3869, 36.2142], 'Первомайский': [49.3869, 36.2142], 'Оріхів': [47.5677, 35.7849], 'Orikhiv': [47.5677, 35.7849], 'Orekhov': [47.5677, 35.7849], 'Орехов': [47.5677, 35.7849], 'Шевченкове': [50.5624, 30.0925], 'Shevchenkove': [50.5624, 30.0925], 'Shevchenkovo': [49.6939, 37.1794], 'Шевченково': [49.6939, 37.1794], 'Щастя': [48.7381, 39.2311], 'Shchastia': [48.7381, 39.2311], 'Schastya': [48.7381, 39.2311], 'Schastie': [48.7381, 39.2311], 'Schastye': [48.7381, 39.2311], 'Счастье': [48.7381, 39.2311], 'Опішня': [49.9564, 34.612], 'Opishnia': [49.9564, 34.612], 'Opishnya': [49.9564, 34.612], 'Oposhnia': [49.9564, 34.612], 'Oposhnya': [49.9564, 34.612], 'Опошня': [49.9564, 34.612], 'Іловайськ': [47.9275, 38.2034], 'Ilovaisk': [47.9275, 38.2034], 'Ilovaysk': [47.9275, 38.2034], 'Illovaisk': [47.9275, 38.2034], 'Illovaysk': [47.9275, 38.2034], 'Иловайск': [47.9275, 38.2034], 'Вишневе': [50.3833, 30.3667], 'Vyshneve': [50.3833, 30.3667], 'Vishnevoe': [50.3833, 30.3667], 'Vishnevoye': [50.3833, 30.3667], 'Vishnyovoye': [50.3833, 30.3667], 'Вишнёвое': [50.3833, 30.3667], 'Краснопілля': [50.7732, 35.2717], 'Krasnopillia': [50.7732, 35.2717], 'Krasnopillya': [50.7732, 35.2717], 'Krasnopolie': [50.7732, 35.2717], 'Krasnopolye': [50.7732, 35.2717], 'Краснополье': [50.7732, 35.2717], 'Українськ': [48.1, 37.3667], 'Ukrainsk': [48.1, 37.3667], 'Ukrayinsk': [48.1, 37.3667], 'Украинск': [48.1, 37.3667], 'Білогородка': [50.4, 30.2167], 'Bilohorodka': [50.4, 30.2167], 'Bilogorodka': [50.4, 30.2167], 'Belogorodka': [50.4, 30.2167], 'Белогородка': [50.4, 30.2167], 'Косів': [48.315, 25.0953], 'Kosiv': [48.315, 25.0953], 'Kosov': [48.315, 25.0953], 'Косов': [48.315, 25.0953], 'Kosiw': [48.315, 25.0953], 'Довжанськ': [48.0846, 39.6516], 'Dovzhansk': [48.0846, 39.6516], 'Свердловськ': [48.0846, 39.6516], 'Свердловск': [48.0846, 39.6516], 'Sverdlovsk': [48.0846, 39.6516], 'Кременець': [50.1081, 25.7275], 'Kremenets': [50.1081, 25.7275], 'Кременец': [50.1081, 25.7275], "Крем'янець": [50.1081, 25.7275], 'Krzemieniec': [50.1081, 25.7275], 'Брянка': [48.5133, 38.6431], 'Brianka': [48.5133, 38.6431], 'Bryanka': [48.5133, 38.6431], 'Клавдієво-Тарасове': [50.5825, 30.0053], 'Klavdiievo-Tarasove': [50.5825, 30.0053], 'Klavdiyevo-Tarasove': [50.5825, 30.0053], 'Klavdiyevo-Tarasovo': [50.5825, 30.0053], 'Клавдиево-Тарасово': [50.5825, 30.0053], 'Королево': [48.15, 23.1333], 'Korolevo': [48.15, 23.1333], 'Koroliovo': [48.15, 23.1333], 'Korolyovo': [48.15, 23.1333], 'Королёво': [48.15, 23.1333], 'Kráľovec nad Tisou': [48.15, 23.1333], 'Királyháza': [48.15, 23.1333], 'Craia': [48.15, 23.1333], 'Рокитне': [49.6897, 30.4751], 'Rokytne': [49.6897, 30.4751], 'Rakytne': [49.6897, 30.4751], 'Rakitnoe': [49.6897, 30.4751], 'Ракитное': [49.6897, 30.4751], 'Суми': [50.9068, 34.7992], 'Sumy': [50.9068, 34.7992], 'Sumi': [50.9068, 34.7992], 'Symi': [50.9068, 34.7992], 'Сумы': [50.9068, 34.7992], 'Бахчисарай': [44.7528, 33.8608], 'Bakhchysarai': [44.7528, 33.8608], 'Bakhchisarai': [44.7528, 33.8608], 'Bahçesaray': [44.7528, 33.8608], 'Bağçasaray': [44.7528, 33.8608], 'Bāghche Sarāy': [44.7528, 33.8608], 'Тисмениця': [48.9008, 24.8492], 'Tysmenytsia': [48.9008, 24.8492], 'Tysmenitsa': [48.9008, 24.8492], 'Тысменица': [48.9008, 24.8492], 'Tyśmienica': [48.9008, 24.8492], 'Південне': [49.8849, 36.0687], 'Pivdenne': [49.8849, 36.0687], 'Pivdennoe': [49.8849, 36.0687], 'Пивденное': [49.8849, 36.0687], 'Гнівань': [49.0833, 28.35], 'Hnivan': [49.0833, 28.35], 'Gnivan': [49.0833, 28.35], 'Гнивань': [49.0833, 28.35], 'Горностаївка': [47.007, 33.728], 'Hornostaivka': [47.007, 33.728], 'Gornostayevka': [47.007, 33.728], 'Gornostaevka': [47.007, 33.728], 'Горностаевка': [47.007, 33.728], 'Верхівцеве': [48.4812, 34.2488], 'Verkhivtseve': [48.4812, 34.2488], 'Verkhovtsevo': [48.4812, 34.2488], 'Верховцево': [48.4812, 34.2488], 'Хотин': [48.5078, 26.486], 'Khotyn': [48.5078, 26.486], 'Khotin': [48.5078, 26.486], 'Chocim': [48.5078, 26.486], 'Chotyn': [48.5078, 26.486], 'Гребінка': [50.118, 32.4396], 'Hrebinka': [50.118, 32.4396], 'Grebenka': [50.118, 32.4396], 'Гребёнка': [50.118, 32.4396], 'Скадовськ': [46.1167, 32.9167], 'Skadovsk': [46.1167, 32.9167], 'Скадовск': [46.1167, 32.9167], 'Харцизьк': [48.0333, 38.15], 'Khartsyzk': [48.0333, 38.15], 'Khartsyzsk': [48.0333, 38.15], 'Харцызск': [48.0333, 38.15], 'Одеса': [46.4775, 30.7326], 'Odesa': [46.4775, 30.7326], 'Odessa': [46.4775, 30.7326], 'Одесса': [46.4775, 30.7326], 'Новий Розділ': [49.4703, 24.13], 'Novyi Rozdil': [49.4703, 24.13], 'Novyi Rozdol': [49.4703, 24.13], 'Noviy Rozdol': [49.4703, 24.13], 'Новый Роздол': [49.4703, 24.13], 'Nowy Rozdół': [49.4703, 24.13], 'Стрий': [49.25, 23.85], 'Stryi': [49.25, 23.85], 'Стрый': [49.25, 23.85], 'Stryj': [49.25, 23.85], 'Лисичанськ': [48.9169, 38.4306], 'Lysychansk': [48.9169, 38.4306], 'Lisichansk': [48.9169, 38.4306], 'Лисичанск': [48.9169, 38.4306], 'Зорія': [45.9917, 29.6989], 'Zoria': [45.9917, 29.6989], 'Zorya': [45.9917, 29.6989], 'Zarya': [45.9917, 29.6989], 'Заря': [45.9917, 29.6989], "Кам'янець-Подільський": [48.6806, 26.5806], 'Kamianets-Podilskyi': [48.6806, 26.5806], 'Kamenets-Podolskiy': [48.6806, 26.5806], 'Kamyanets-Podilskyi': [48.6806, 26.5806], 'Kamianets-Podolskyi': [48.6806, 26.5806], 'Каменец-Подольский': [48.6806, 26.5806], 'Kamieniec Podolski': [48.6806, 26.5806], 'Camenița': [48.6806, 26.5806], 'Лебедин': [50.5831, 34.4823], 'Lebedyn': [50.5831, 34.4823], 'Lebedin': [50.5831, 34.4823], 'Самбір': [49.5167, 23.2], 'Sambir': [49.5167, 23.2], 'Sambor': [49.5167, 23.2], 'Самбор': [49.5167, 23.2], 'Жовті Води': [48.35, 33.5167], 'Zhovti Vody': [48.35, 33.5167], 'Zheltie Vody': [48.35, 33.5167], 'Zheltye Vody': [48.35, 33.5167], 'Жёлтые Воды': [48.35, 33.5167], 'Шаргород': [48.75, 28.0833], 'Sharhorod': [48.75, 28.0833], 'Shargorod': [48.75, 28.0833], 'Szarogród': [48.75, 28.0833], 'Новогуйвинське': [50.2022, 28.685], 'Novohuivynske': [50.2022, 28.685], 'Novohuivynskoe': [50.2022, 28.685], 'Novoguyvinskoe': [50.2022, 28.685], 'Новогуйвинское': [50.2022, 28.685], 'Хмельницький': [49.42, 27], 'Khmelnytskyi': [49.42, 27], 'Khmelnitsky': [49.42, 27], 'Хмельницкий': [49.42, 27], 'Чорнобаївка': [46.7004, 32.5477], 'Chornobaivka': [46.7004, 32.5477], 'Chornobayivka': [46.7004, 32.5477], 'Chernobaevka': [46.7004, 32.5477], 'Chernobayevka': [46.7004, 32.5477], 'Chernobaivka': [46.7004, 32.5477], 'Іларіонове': [48.4059, 35.2734], 'Ilarionove': [48.4059, 35.2734], 'Ilarionovo': [48.4059, 35.2734], 'Illarionovo': [48.4059, 35.2734], 'Illarionove': [48.4059, 35.2734], 'Илларионово': [48.4059, 35.2734], 'Буча': [50.5464, 30.235], 'Bucha': [50.5464, 30.235], 'Butcha': [50.5464, 30.235], 'Андрушівка': [50.0167, 29.0167], 'Andrushivka': [50.0167, 29.0167], 'Andrushevka': [50.0167, 29.0167], 'Андрушевка': [50.0167, 29.0167], 'Andruszówka': [50.0167, 29.0167], 'Вовчанськ': [50.2881, 36.9461], 'Vovchansk': [50.2881, 36.9461], 'Volchansk': [50.2881, 36.9461], 'Волчанск': [50.2881, 36.9461], 'Богданівка': [48.5015, 36.11], 'Bohdanivka': [48.5015, 36.11], 'Bogdanovka': [48.5015, 36.11], 'Богдановка': [48.5015, 36.11], 'Свеса': [51.9481, 33.9367], 'Svesa': [51.9481, 33.9367], 'Svessa': [51.9481, 33.9367], 'Свесса': [51.9481, 33.9367], 'Вишгород': [50.5833, 30.5], 'Vyshhorod': [50.5833, 30.5], 'Vyshgorod': [50.5833, 30.5], 'Вышгород': [50.5833, 30.5], 'Біловодськ': [49.1992, 39.5756], 'Bilovodsk': [49.1992, 39.5756], 'Belovodsk': [49.1992, 39.5756], 'Беловодск': [49.1992, 39.5756], 'Яремче': [48.4536, 24.5564], 'Yaremche': [48.4536, 24.5564], 'Jaremcze': [48.4536, 24.5564], 'Jaremcza': [48.4536, 24.5564], 'Вільшани': [50.0533, 35.8847], 'Vilshany': [50.0533, 35.8847], 'Vilshani': [50.0533, 35.8847], 'Olshany': [50.0533, 35.8847], 'Ольшаны': [50.0533, 35.8847], 'Краматорськ': [48.7208, 37.5556], 'Kramatorsk': [48.7208, 37.5556], 'Краматорск': [48.7208, 37.5556], 'Кушугум': [47.7106, 35.2192], 'Kushuhum': [47.7106, 35.2192], 'Kushugum': [47.7106, 35.2192], 'Моршин': [49.155, 23.8725], 'Morshyn': [49.155, 23.8725], 'Morshin': [49.155, 23.8725], 'Morszyn-Zdrój': [49.155, 23.8725], 'Узин': [49.8242, 30.4425], 'Uzyn': [49.8242, 30.4425], 'Uzin': [49.8242, 30.4425], 'Ялта': [44.4994, 34.1553], 'Yalta': [44.4994, 34.1553], 'Ialta': [44.4994, 34.1553], 'Jałta': [44.4994, 34.1553], 'Yalita': [44.4994, 34.1553], 'Лисянка': [49.2542, 30.8207], 'Lysianka': [49.2542, 30.8207], 'Lysyanka': [49.2542, 30.8207], 'Лысянка': [49.2542, 30.8207], 'Гуляйполе': [47.6644, 36.2632], 'Huliaipole': [47.6644, 36.2632], 'Hulaypole': [47.6644, 36.2632], 'Hulyaypole': [47.6644, 36.2632], 'Guliapole': [47.6644, 36.2632], 'Gulaypole': [47.6644, 36.2632], 'Пологи': [47.4796, 36.2611], 'Polohy': [47.4796, 36.2611], 'Polohi': [47.4796, 36.2611], 'Pologi': [47.4796, 36.2611], 'Pology': [47.4796, 36.2611], 'Чубарівка': [47.4796, 36.2611], 'Чубаровка': [47.4796, 36.2611], 'Chubarivka': [47.4796, 36.2611], 'Chubarovka': [47.4796, 36.2611], 'Отинія': [48.7339, 24.8569], 'Otyniia': [48.7339, 24.8569], 'Ottynia': [48.7339, 24.8569], 'Otyniya': [48.7339, 24.8569], 'Otynya': [48.7339, 24.8569], 'Otinya': [48.7339, 24.8569], 'Отыния': [48.7339, 24.8569], 'Oteniia': [48.7339, 24.8569], 'Тетіїв': [49.3708, 29.69], 'Tetiiv': [49.3708, 29.69], 'Tetiev': [49.3708, 29.69], 'Тетиев': [49.3708, 29.69], 'Добротвір': [50.2056, 24.3864], 'Dobrotvir': [50.2056, 24.3864], 'Dobrotvor': [50.2056, 24.3864], 'Добротвор': [50.2056, 24.3864], 'Dobrotwór': [50.2056, 24.3864], 'Ширяєве': [47.3878, 30.1911], 'Shyriaieve': [47.3878, 30.1911], 'Shiriaievo': [47.3878, 30.1911], 'Shiryaevo': [47.3878, 30.1911], 'Ширяево': [47.3878, 30.1911], 'Горенка': [50.5569, 30.3175], 'Horenka': [50.5569, 30.3175], 'Gorenka': [50.5569, 30.3175], 'Станиця Луганська': [48.6667, 39.4667], 'Stanytsia Luhanska': [48.6667, 39.4667], 'Stanitsa Luganskaya': [48.6667, 39.4667], 'Станица Луганская': [48.6667, 39.4667], 'Ананьїв': [47.7225, 29.96], 'Ananiv': [47.7225, 29.96], 'Ananiev': [47.7225, 29.96], 'Ананьев': [47.7225, 29.96], 'Яни Капу': [45.9675, 33.8003], 'Yany Kapu': [45.9675, 33.8003], 'Красноперекопськ': [45.9675, 33.8003], 'Krasnoperekopsk': [45.9675, 33.8003], 'Yañı Qapı': [45.9675, 33.8003], 'Марганець': [47.648, 34.6167], 'Marhanets': [47.648, 34.6167], 'Marganets': [47.648, 34.6167], 'Марганец': [47.648, 34.6167], 'Баришівка': [50.3703, 31.3286], 'Baryshivka': [50.3703, 31.3286], 'Baryshevka': [50.3703, 31.3286], 'Барышевка': [50.3703, 31.3286], 'Волноваха': [47.5725, 31.3119], 'Volnovakha': [47.5725, 31.3119], 'Volnavakha': [47.5725, 31.3119], 'Volnovaha': [47.5725, 31.3119], 'Першотравенськ': [48.3464, 36.4044], 'Pershotravensk': [48.3464, 36.4044], 'Pervomaysk': [48.3464, 36.4044], 'Pervomaisk': [48.3464, 36.4044], 'Першотравенск': [48.3464, 36.4044], 'Первомайск': [48.3464, 36.4044], 'Масандра': [44.5167, 34.1833], 'Masandra': [44.5167, 34.1833], 'Massandra': [44.5167, 34.1833], 'Массандра': [44.5167, 34.1833], 'Могилів-Подільський': [48.45, 27.7833], 'Mohyliv-Podilskyi': [48.45, 27.7833], 'Mogilev-Podolsky': [48.45, 27.7833], 'Mogilev-Podolskyi': [48.45, 27.7833], 'Mogilev-Podolskiy': [48.45, 27.7833], 'Могилёв-Подольский': [48.45, 27.7833], 'Mohylów Podolski': [48.45, 27.7833], 'Moghilău': [48.45, 27.7833], 'Movilău': [48.45, 27.7833], 'Липовець': [49.2161, 29.0561], 'Lypovets': [49.2161, 29.0561], 'Lipovets': [49.2161, 29.0561], 'Липовец': [49.2161, 29.0561], 'Переяслав': [50.065, 31.445], 'Pereiaslav': [50.065, 31.445], 'Pereyaslav': [50.065, 31.445], 'Переяслав-Хмельницький': [50.065, 31.445], 'Переяслав-Хмельницкий': [50.065, 31.445], 'Pereyaslav-Khmelnytskyy': [50.065, 31.445], 'Pereyaslav-Khmelnitsky': [50.065, 31.445], 'Pereyaslav-Khmelnitskiy': [50.065, 31.445], 'Дрогобич': [49.35, 23.5], 'Drohobych': [49.35, 23.5], 'Drogobych': [49.35, 23.5], 'Drogobich': [49.35, 23.5], 'Дрогобыч': [49.35, 23.5], 'Drohobycz': [49.35, 23.5], "Прип'ять": [51.406, 30.044], 'Prypyat': [51.406, 30.044], 'Pripyat': [51.406, 30.044], 'Pripiat': [51.406, 30.044], 'Припять': [51.406, 30.044], 'Братське': [47.8636, 31.5781], 'Bratske': [47.8636, 31.5781], 'Bratskoe': [47.8636, 31.5781], 'Братское': [47.8636, 31.5781], 'Луганськ': [48.5833, 39.3333], 'Luhansk': [48.5833, 39.3333], 'Lugansk': [48.5833, 39.3333], 'Луганск': [48.5833, 39.3333], 'Voroshilovgrad': [48.5833, 39.3333], 'Іванівка': [48.2325, 38.9525], 'Ivanivka': [48.2325, 38.9525], 'Ivanovka': [48.2325, 38.9525], 'Ивановка': [48.2325, 38.9525], 'Iwanivka': [48.2325, 38.9525], 'Долинська': [48.1111, 32.7648], 'Dolynska': [48.1111, 32.7648], 'Dolinskaya': [48.1111, 32.7648], 'Долинская': [48.1111, 32.7648], 'Ізяслав': [50.1167, 26.8], 'Iziaslav': [50.1167, 26.8], 'Zaslav': [50.1167, 26.8], 'Izyaslav': [50.1167, 26.8], 'Изяслав': [50.1167, 26.8], 'Свалява': [48.5472, 22.9861], 'Svaliava': [48.5472, 22.9861], 'Svalyava': [48.5472, 22.9861], 'Szolyva': [48.5472, 22.9861], 'Svaľava': [48.5472, 22.9861], 'Макіївка': [48.0556, 37.9611], 'Makiivka': [48.0556, 37.9611], 'Makiyivka': [48.0556, 37.9611], 'Makeyevka': [48.0556, 37.9611], 'Makeievka': [48.0556, 37.9611], 'Макеевка': [48.0556, 37.9611], 'Дмитріївськ': [48.0556, 37.9611], 'Дмитриевск': [48.0556, 37.9611], 'Dmytriivsk': [48.0556, 37.9611], 'Dmitrievsk': [48.0556, 37.9611], 'Дмитріївськ-Сталінськ': [48.0556, 37.9611], 'Дмитриевск-Сталинск': [48.0556, 37.9611], 'Dmitrievsk-Stalinsk': [48.0556, 37.9611], 'Стара Синява': [49.6, 27.6167], 'Stara Syniava': [49.6, 27.6167], 'Stara Siniava': [49.6, 27.6167], 'Staraya Sinyava': [49.6, 27.6167], 'Старая Синява': [49.6, 27.6167], 'Золочів': [49.8075, 24.9031], 'Zolochiv': [49.8075, 24.9031], 'Zolochev': [49.8075, 24.9031], 'Золочев': [49.8075, 24.9031], 'Złoczów': [49.8075, 24.9031], 'Solotschiw': [49.8075, 24.9031], 'Прилуки': [50.6, 32.4], 'Pryluky': [50.6, 32.4], 'Priluki': [50.6, 32.4], 'Ізюм': [49.2238, 37.2915], 'Izium': [49.2238, 37.2915], 'Izum': [49.2238, 37.2915], 'Izyum': [49.2238, 37.2915], 'Изюм': [49.2238, 37.2915], 'Калуш': [49.0442, 24.3597], 'Kalush': [49.0442, 24.3597], 'Kaluzh': [49.0442, 24.3597], 'Kałusz': [49.0442, 24.3597], 'Городок': [49.1667, 26.5667], 'Horodok': [49.1667, 26.5667], 'Gorodok': [49.1667, 26.5667], 'Старобільськ': [49.2667, 38.9167], 'Starobilsk': [49.2667, 38.9167], 'Starobelsk': [49.2667, 38.9167], 'Старобельск': [49.2667, 38.9167], 'Калинівка': [49.4472, 28.5231], 'Kalynivka': [49.4472, 28.5231], 'Kalinovka': [49.4472, 28.5231], 'Калиновка': [49.4472, 28.5231], 'Kalinówka': [49.4472, 28.5231], 'Мукачеве': [48.4414, 22.7136], 'Mukacheve': [48.4414, 22.7136], 'Mukachev': [48.4414, 22.7136], 'Mukachevo': [48.4414, 22.7136], 'Мукачево': [48.4414, 22.7136], 'Mukaczewo': [48.4414, 22.7136], 'Muncaciu': [48.4414, 22.7136], 'Munkács': [48.4414, 22.7136], 'Mukačevo': [48.4414, 22.7136], 'Munkatsch': [48.4414, 22.7136], 'Тростянець': [48.512, 29.2137], 'Trostianets': [48.512, 29.2137], 'Trostyanets': [48.512, 29.2137], 'Тростянец': [48.512, 29.2137], 'Дружківка': [48.6203, 37.5278], 'Druzhkivka': [48.6203, 37.5278], 'Druzhkovka': [48.6203, 37.5278], 'Дружковка': [48.6203, 37.5278], 'Обухівка': [48.5442, 34.8664], 'Obukhivka': [48.5442, 34.8664], 'Obukhovka': [48.5442, 34.8664], 'Обуховка': [48.5442, 34.8664], 'Привілля': [48.9999, 38.2948], 'Pryvillia': [48.9999, 38.2948], 'Privolie': [48.9999, 38.2948], 'Privolye': [48.9999, 38.2948], 'Приволье': [48.9999, 38.2948], 'Горонда': [48.3786, 22.5681], 'Horonda': [48.3786, 22.5681], 'Goronda': [48.3786, 22.5681], 'Лиманське': [46.6639, 29.9708], 'Lymanske': [46.6639, 29.9708], 'Lymanskoe': [46.6639, 29.9708], 'Limanskoe': [46.6639, 29.9708], 'Лиманское': [46.6639, 29.9708], 'Горлівка': [48.3336, 38.0925], 'Horlivka': [48.3336, 38.0925], 'Gorlovka': [48.3336, 38.0925], 'Горловка': [48.3336, 38.0925], 'Gorlowka': [48.3336, 38.0925], 'Лохвиця': [50.361, 33.2652], 'Lokhvytsia': [50.361, 33.2652], 'Lokhvytsya': [50.361, 33.2652], 'Lokhvitsa': [50.361, 33.2652], 'Лохвица': [50.361, 33.2652], 'Недригайлів': [50.8332, 33.8799], 'Nedryhailiv': [50.8332, 33.8799], 'Nedrygailiv': [50.8332, 33.8799], 'Nedrigaylov': [50.8332, 33.8799], 'Недригайлов': [50.8332, 33.8799], 'Каховка': [46.8, 33.4667], 'Kakhovka': [46.8, 33.4667], 'Kahovka': [46.8, 33.4667], 'Городниця': [50.8094, 27.3183], 'Horodnytsia': [50.8094, 27.3183], 'Gorodnytsia': [50.8094, 27.3183], 'Gorodnitsa': [50.8094, 27.3183], 'Городница': [50.8094, 27.3183], 'Великі Мости': [50.24, 24.1394], 'Velyki Mosty': [50.24, 24.1394], 'Velikie Mosty': [50.24, 24.1394], 'Великие Мосты': [50.24, 24.1394], 'Mosty Wielkie': [50.24, 24.1394], 'Городня': [51.8833, 31.5833], 'Horodnia': [51.8833, 31.5833], 'Gorodnia': [51.8833, 31.5833], 'Gorodnya': [51.8833, 31.5833], 'Подільськ': [47.7419, 29.535], 'Podilsk': [47.7419, 29.535], 'Podolsk': [47.7419, 29.535], 'Подольск': [47.7419, 29.535], 'Котовськ': [47.7419, 29.535], 'Котовск': [47.7419, 29.535], 'Kotovsk': [47.7419, 29.535], 'Bârzula': [47.7419, 29.535], 'Bârzu': [47.7419, 29.535], 'Ворзель': [50.55, 30.15], 'Vorzel': [50.55, 30.15], 'Vorsel': [50.55, 30.15], 'Білогірськ': [45.0544, 34.6022], 'Bilohirsk': [45.0544, 34.6022], 'Belogorsk': [45.0544, 34.6022], 'Белогорск': [45.0544, 34.6022], 'Karasubazar': [45.0544, 34.6022], 'Магуш': [47.05, 37.3], 'Manhush': [47.05, 37.3], 'Mangush': [47.05, 37.3], 'Pershotravneve': [47.05, 37.3], 'Pershotravnevoe': [47.05, 37.3], 'Pershotravnevy': [47.05, 37.3], 'Першотравневое': [47.05, 37.3], 'Першотравневый': [47.05, 37.3], 'Малин': [50.7689, 29.27], 'Malyn': [50.7689, 29.27], 'Malin': [50.7689, 29.27], 'Нова Каховка': [46.7667, 33.3667], 'Nova Kakhovka': [46.7667, 33.3667], 'Novaya Kakhovka': [46.7667, 33.3667], 'Новая Каховка': [46.7667, 33.3667], 'Кременчук': [49.0775, 33.4239], 'Kremenchuk': [49.0775, 33.4239], 'Kremenchug': [49.0775, 33.4239], 'Жмеринка': [49.0425, 28.0992], 'Zhmerynka': [49.0425, 28.0992], 'Zhmerinka': [49.0425, 28.0992], 'Żmerynka': [49.0425, 28.0992], 'Șmerinca': [49.0425, 28.0992], 'Козелець': [50.9164, 31.1147], 'Kozelets': [50.9164, 31.1147], 'Kozhelets': [50.9164, 31.1147], 'Козелец': [50.9164, 31.1147], 'Kozelez': [50.9164, 31.1147], 'Петропавлiвка': [48.4549, 36.4394], 'Petropavlivka': [48.4549, 36.4394], 'Petropavlovka': [48.4549, 36.4394], 'Петропавловка': [48.4549, 36.4394], 'Брюховичі': [49.9, 23.9436], 'Briukhovychi': [49.9, 23.9436], 'Bryukhovichi': [49.9, 23.9436], 'Bryukhovychi': [49.9, 23.9436], 'Briuhovychi': [49.9, 23.9436], 'Київ': [50.4501, 30.5234], 'Kyiv': [50.4501, 30.5234], 'Kiev': [50.4501, 30.5234], 'Kiyv': [50.4501, 30.5234], 'Kyyv': [50.4501, 30.5234], 'Kyyiv': [50.4501, 30.5234], 'Киев': [50.4501, 30.5234], 'Kijów': [50.4501, 30.5234], 'Kyjiw': [50.4501, 30.5234], 'Kiew': [50.4501, 30.5234], 'Саки': [45.1336, 33.5772], 'Saky': [45.1336, 33.5772], 'Saki': [45.1336, 33.5772], 'Saq': [45.1336, 33.5772], 'Гоща': [50.5986, 26.6753], 'Hoshcha': [50.5986, 26.6753], 'Goscha': [50.5986, 26.6753], 'Hoszcza': [50.5986, 26.6753], 'Слатине': [50.2161, 36.1525], 'Slatyne': [50.2161, 36.1525], 'Slatino': [50.2161, 36.1525], 'Слатино': [50.2161, 36.1525], "Знам'янка": [48.7136, 32.6733], 'Znamianka': [48.7136, 32.6733], 'Znamenka': [48.7136, 32.6733], 'Знаменка': [48.7136, 32.6733], 'Васильків': [50.1775, 30.3217], 'Vasylkiv': [50.1775, 30.3217], 'Vasilkov': [50.1775, 30.3217], 'Васильков': [50.1775, 30.3217], 'Підгородне': [48.5737, 35.0966], 'Pidhorodne': [48.5737, 35.0966], 'Pidgorodne': [48.5737, 35.0966], 'Podgorodnoe': [48.5737, 35.0966], 'Подгородное': [48.5737, 35.0966], 'Верховина': [48.1517, 24.8136], 'Verkhovyna': [48.1517, 24.8136], 'Verkhovina': [48.1517, 24.8136], 'Żabie': [48.1517, 24.8136], 'Вільногірськ': [48.4824, 34.0189], 'Vilnohirsk': [48.4824, 34.0189], 'Vilnogirsk': [48.4824, 34.0189], 'Volnogorsk': [48.4824, 34.0189], 'Вольногорск': [48.4824, 34.0189], 'Немішаєве': [50.5624, 30.0925], 'Nemishaieve': [50.5624, 30.0925], 'Nemishayeve': [50.5624, 30.0925], 'Nemysheev': [50.5624, 30.0925], 'Nemeshaevo': [50.5624, 30.0925], 'Nemeshayevo': [50.5624, 30.0925], 'Немешаево': [50.5624, 30.0925], 'Шевченковое': [50.5624, 30.0925], 'Shevchenkovoe': [50.5624, 30.0925], 'Євпаторія': [45.2, 33.3583], 'Yevpatoriia': [45.2, 33.3583], 'Yevpatoria': [45.2, 33.3583], 'Yevpatoriya': [45.2, 33.3583], 'Evpatoria': [45.2, 33.3583], 'Евпатория': [45.2, 33.3583], 'Віньківці': [49.0333, 27.2333], 'Vinkivtsi': [49.0333, 27.2333], 'Vinkovtsy': [49.0333, 27.2333], 'Виньковцы': [49.0333, 27.2333], 'Немирів': [48.9794, 28.8439], 'Nemyriv': [48.9794, 28.8439], 'Nemirov': [48.9794, 28.8439], 'Nemiroff': [48.9794, 28.8439], 'Немиров': [48.9794, 28.8439], 'Niemirów': [48.9794, 28.8439], "Великі Ком'яти": [48.2386, 22.9789], 'Velyki Komiaty': [48.2386, 22.9789], 'Velyki Komyaty': [48.2386, 22.9789], 'Velikie Komyaty': [48.2386, 22.9789], 'Великие Комяты': [48.2386, 22.9789], 'Рахів': [48.05, 24.2167], 'Rakhiv': [48.05, 24.2167], 'Rakhov': [48.05, 24.2167], 'Рахов': [48.05, 24.2167], 'Rahó': [48.05, 24.2167], 'Rahău': [48.05, 24.2167], 'Оленівка': [47.8313, 37.6423], 'Olenivka': [47.8313, 37.6423], 'Olenovka': [47.8313, 37.6423], 'Elenovka': [47.8313, 37.6423], 'Еленовка': [47.8313, 37.6423], 'Оленовка': [47.8313, 37.6423], 'Іванків': [50.9333, 29.9], 'Ivankiv': [50.9333, 29.9], 'Ivankov': [50.9333, 29.9], 'Иванков': [50.9333, 29.9], 'Ivanquiv': [50.9333, 29.9], 'Томаківка': [47.8163, 34.7471], 'Tomakivka': [47.8163, 34.7471], 'Tomakovka': [47.8163, 34.7471], 'Томаковка': [47.8163, 34.7471], 'Любешів': [51.7631, 25.4992], 'Liubeshiv': [51.7631, 25.4992], 'Liubeshov': [51.7631, 25.4992], 'Lyubeshov': [51.7631, 25.4992], 'Любешов': [51.7631, 25.4992], 'Lubieszów': [51.7631, 25.4992], 'Любяшоў': [51.7631, 25.4992], 'Болехів': [49.0669, 23.8514], 'Bolekhiv': [49.0669, 23.8514], 'Bolehiv': [49.0669, 23.8514], 'Bolechiw': [49.0669, 23.8514], 'Болехов': [49.0669, 23.8514], 'Bolechów': [49.0669, 23.8514], 'Степанівка': [50.943, 34.6334], 'Stepanivka': [50.943, 34.6334], 'Stepanovka': [50.943, 34.6334], 'Степановка': [50.943, 34.6334], 'Мелітополь': [46.8333, 35.3667], 'Melitopol': [46.8333, 35.3667], 'Мелитополь': [46.8333, 35.3667], 'Melitopoli': [46.8333, 35.3667], 'Тлумач': [48.8669, 25.0012], 'Tlumach': [48.8669, 25.0012], 'Tovmach': [48.8669, 25.0012], 'Товмач': [48.8669, 25.0012], 'Tłumacz': [48.8669, 25.0012], 'Гвардійське': [48.7328, 35.311], 'Hvardiiske': [48.7328, 35.311], 'Hvardiyske': [48.7328, 35.311], 'Гвардейское': [48.7328, 35.311], 'Перегінське': [48.8103, 24.1819], 'Perehinske': [48.8103, 24.1819], 'Pereginske': [48.8103, 24.1819], 'Pereginskoe': [48.8103, 24.1819], 'Перегинское': [48.8103, 24.1819], 'Perehińsko': [48.8103, 24.1819], 'Бердянськ': [46.7598, 36.7845], 'Berdiansk': [46.7598, 36.7845], 'Berdyansk': [46.7598, 36.7845], 'Бердианск': [46.7598, 36.7845], 'Коломия': [48.5167, 25.0333], 'Kolomyia': [48.5167, 25.0333], 'Kolomia': [48.5167, 25.0333], 'Kolomea': [48.5167, 25.0333], 'Kołomyja': [48.5167, 25.0333], 'Colomeea': [48.5167, 25.0333], 'Сєвєродонецьк': [48.95, 38.4833], 'Sievierodonetsk': [48.95, 38.4833], 'Siverskodonetsk': [48.95, 38.4833], 'Syeverodonetsk': [48.95, 38.4833], 'Severodonetsk': [48.95, 38.4833], 'Сіверськодонецьк': [48.95, 38.4833], 'Северодонецк': [48.95, 38.4833], 'Дубове': [48.1781, 23.8863], 'Dubove': [48.1781, 23.8863], 'Dubovoe': [48.1781, 23.8863], 'Дубовое': [48.1781, 23.8863], 'Dombó': [48.1781, 23.8863], 'Delureni': [48.1781, 23.8863], 'Ланівці': [49.8644, 26.0782], 'Lanivtsi': [49.8644, 26.0782], 'Lanovtsy': [49.8644, 26.0782], 'Лановцы': [49.8644, 26.0782], 'Łanowce': [49.8644, 26.0782], 'Кобеляки': [49.1474, 34.1993], 'Kobeliaky': [49.1474, 34.1993], 'Kobelyaky': [49.1474, 34.1993], 'Kobelyaki': [49.1474, 34.1993], 'Кривий Ріг': [47.9086, 33.3433], 'Kryvyi Rih': [47.9086, 33.3433], 'Kryvyi Rig': [47.9086, 33.3433], 'Kriviy Rig': [47.9086, 33.3433], 'Krivoy Rog': [47.9086, 33.3433], 'Кривой Рог': [47.9086, 33.3433], 'Яготин': [50.2569, 31.7817], 'Yahotyn': [50.2569, 31.7817], 'Yagotyn': [50.2569, 31.7817], 'Yagotin': [50.2569, 31.7817], 'Виноградів': [48.1397, 23.0331], 'Vynohradiv': [48.1397, 23.0331], 'Vinogradov': [48.1397, 23.0331], 'Виноградов': [48.1397, 23.0331], 'Nagyszőlős': [48.1397, 23.0331], 'Seleușu Mare': [48.1397, 23.0331], 'Vinohradov': [48.1397, 23.0331], 'Чернівці': [48.2908, 25.9344], 'Chernivtsi': [48.2908, 25.9344], 'Chernyvtsi': [48.2908, 25.9344], 'Черновцы': [48.2908, 25.9344], 'Cernăuți': [48.2908, 25.9344], 'Czernowitz': [48.2908, 25.9344], 'Житомир': [50.2544, 28.6578], 'Zhytomyr': [50.2544, 28.6578], 'Zhitomyr': [50.2544, 28.6578], 'Zhytomir': [50.2544, 28.6578], 'Żytomierz': [50.2544, 28.6578], 'Чортків': [49.0167, 25.8], 'Chortkiv': [49.0167, 25.8], 'Chortkov': [49.0167, 25.8], 'Чортков': [49.0167, 25.8], 'Czortków': [49.0167, 25.8], 'Великий Березний': [48.8922, 22.4586], 'Velykyi Bereznyi': [48.8922, 22.4586], 'Velikiy Bereznyi': [48.8922, 22.4586], 'Великий Березный': [48.8922, 22.4586], 'Veľké Berezné': [48.8922, 22.4586], 'Стрижавка': [49.3103, 28.4808], 'Stryzhavka': [49.3103, 28.4808], 'Strizhavka': [49.3103, 28.4808], 'Strzyżawka': [49.3103, 28.4808], 'Помічна': [48.25, 31.4167], 'Pomichna': [48.25, 31.4167], 'Pomoshnaya': [48.25, 31.4167], 'Pomochnaya': [48.25, 31.4167], 'Помошная': [48.25, 31.4167], 'Криве Озеро': [47.9333, 30.3333], 'Kryve Ozero': [47.9333, 30.3333], 'Krivoe Ozero': [47.9333, 30.3333], 'Krivoye Ozero': [47.9333, 30.3333], 'Кривое Озеро': [47.9333, 30.3333], 'Радивилів': [50.1333, 25.25], 'Radyvyliv': [50.1333, 25.25], 'Radivilov': [50.1333, 25.25], 'Радивилов': [50.1333, 25.25], 'Білокуракине': [49.5167, 38.7167], 'Bilokurakyne': [49.5167, 38.7167], 'Belokurakino': [49.5167, 38.7167], 'Белокуракино': [49.5167, 38.7167], 'Перещепине': [49.0179, 35.3598], 'Pereshchepyne': [49.0179, 35.3598], 'Pereshchepino': [49.0179, 35.3598], 'Перещепино': [49.0179, 35.3598], 'Канів': [49.7447, 31.4558], 'Kaniv': [49.7447, 31.4558], 'Kanev': [49.7447, 31.4558], 'Канев': [49.7447, 31.4558], 'Kaniów': [49.7447, 31.4558], 'Ясногірка': [48.7725, 37.5506], 'Yasnohirka': [48.7725, 37.5506], 'Yasnogorka': [48.7725, 37.5506], 'Ясногорка': [48.7725, 37.5506], 'Донецьк': [48.0089, 37.8042], 'Donetsk': [48.0089, 37.8042], 'Донецк': [48.0089, 37.8042], 'Donieck': [48.0089, 37.8042], 'Aleksandrovka': [48.0089, 37.8042], 'Yuzivka': [48.0089, 37.8042], 'Stalino': [48.0089, 37.8042], 'Хрустальний': [48.1214, 38.9453], 'Khrustalnyi': [48.1214, 38.9453], 'Hrustalnyi': [48.1214, 38.9453], 'Хрустальный': [48.1214, 38.9453], 'Криндачівка': [48.1214, 38.9453], 'Kryndachivka': [48.1214, 38.9453], 'Krindachyovka': [48.1214, 38.9453], 'Красный Луч': [48.1214, 38.9453], 'Krasnyi Luch': [48.1214, 38.9453], 'Нікополь': [47.5772, 34.3575], 'Nikopol': [47.5772, 34.3575], 'Никополь': [47.5772, 34.3575], 'Νικόπολις': [47.5772, 34.3575], 'Печеніжин': [48.51, 24.8892], 'Pechenizhyn': [48.51, 24.8892], 'Pechenezhin': [48.51, 24.8892], 'Печенежин': [48.51, 24.8892], 'Peczeniżyn': [48.51, 24.8892], 'Магдалинiвка': [48.9147, 34.9154], 'Mahdalynivka': [48.9147, 34.9154], 'Magdalynivka': [48.9147, 34.9154], 'Magdalinovka': [48.9147, 34.9154], 'Магдалиновка': [48.9147, 34.9154], 'Костопіль': [50.8833, 26.4431], 'Kostopil': [50.8833, 26.4431], 'Kostopol': [50.8833, 26.4431], 'Костополь': [50.8833, 26.4431], 'Безлюдівка': [49.8694, 36.2719], 'Bezliudivka': [49.8694, 36.2719], 'Bezludivka': [49.8694, 36.2719], 'Безлюдовка': [49.8694, 36.2719], 'Чемерівці': [49, 26.3667], 'Chemerivtsi': [49, 26.3667], 'Chemerovtsi': [49, 26.3667], 'Chemerovtsy': [49, 26.3667], 'Чемеровцы': [49, 26.3667], 'Печеніги': [49.8647, 36.9364], 'Pechenihy': [49.8647, 36.9364], 'Pechenegi': [49.8647, 36.9364], 'Печенеги': [49.8647, 36.9364], 'Ровеньки': [48.0833, 39.3667], 'Rovenky': [48.0833, 39.3667], 'Rovenki': [48.0833, 39.3667], 'Сутиски': [49.0422, 28.4208], 'Sutysky': [49.0422, 28.4208], 'Sutiski': [49.0422, 28.4208], 'Селидове': [48.15, 37.3], 'Selydove': [48.15, 37.3], 'Selidove': [48.15, 37.3], 'Selidovo': [48.15, 37.3], 'Селидово': [48.15, 37.3], 'Селидівка': [48.15, 37.3], 'Селидовка': [48.15, 37.3], 'Selydivka': [48.15, 37.3], 'Мирноград': [48.3022, 37.2614], 'Myrnohrad': [48.3022, 37.2614], 'Mirnograd': [48.3022, 37.2614], 'Димитров': [48.3022, 37.2614], 'Dymytrov': [48.3022, 37.2614], 'Dimitrov': [48.3022, 37.2614], 'Буштино': [48.0503, 23.4894], 'Bushtyno': [48.0503, 23.4894], 'Bushtino': [48.0503, 23.4894], 'Буштыно': [48.0503, 23.4894], 'Bustyaháza': [48.0503, 23.4894], 'Bustea': [48.0503, 23.4894], 'Buštín': [48.0503, 23.4894], 'Енергодар': [47.4989, 34.6558], 'Enerhodar': [47.4989, 34.6558], 'Energodar': [47.4989, 34.6558], 'Энергодар': [47.4989, 34.6558], 'Коцюбинське': [50.4883, 30.3331], 'Kotsiubynske': [50.4883, 30.3331], 'Kotsyubynske': [50.4883, 30.3331], 'Kotsubinskoe': [50.4883, 30.3331], 'Kotsiubinskoe': [50.4883, 30.3331], 'Коцюбинское': [50.4883, 30.3331], 'Южне': [46.6222, 31.1008], 'Yuzhne': [46.6222, 31.1008], 'Yuzhny': [46.6222, 31.1008], 'Yuzhniy': [46.6222, 31.1008], 'Yuzhnoe': [46.6222, 31.1008], 'Yuzhnoye': [46.6222, 31.1008], 'Южний': [46.6222, 31.1008], 'Южное': [46.6222, 31.1008], 'Чернігів': [51.4939, 31.2947], 'Chernihiv': [51.4939, 31.2947], 'Chernigov': [51.4939, 31.2947], 'Chernigiv': [51.4939, 31.2947], 'Чернигов': [51.4939, 31.2947], 'Czernihów': [51.4939, 31.2947], 'Tschernihiv': [51.4939, 31.2947], 'Czerniusza': [51.4939, 31.2947], 'Борова': [50.1733, 30.1039], 'Borova': [50.1733, 30.1039], 'Borovaya': [50.1733, 30.1039], 'Боровая': [50.1733, 30.1039], 'Межова': [48.2563, 36.7313], 'Mezhova': [48.2563, 36.7313], 'Mezhevaya': [48.2563, 36.7313], 'Mezhevaia': [48.2563, 36.7313], 'Межевая': [48.2563, 36.7313], 'Обухів': [50.1, 30.6167], 'Obukhiv': [50.1, 30.6167], 'Obukhov': [50.1, 30.6167], 'Obuhiv': [50.1, 30.6167], 'Обухов': [50.1, 30.6167], 'Вороновиця': [49.0989, 28.6831], 'Voronovytsia': [49.0989, 28.6831], 'Voronovitsa': [49.0989, 28.6831], 'Вороновица': [49.0989, 28.6831], 'Синельникове': [48.3178, 35.5119], 'Synelnykove': [48.3178, 35.5119], 'Sinelnikovo': [48.3178, 35.5119], 'Синельниково': [48.3178, 35.5119], 'Новомиколаївка': [47.9779, 35.9099], 'Novomykolaivka': [47.9779, 35.9099], 'Novomykolayivka': [47.9779, 35.9099], 'Novomikolayivka': [47.9779, 35.9099], 'Novonikolaevka': [47.9779, 35.9099], 'Новониколаевка': [47.9779, 35.9099], 'Городенка': [48.6675, 25.5003], 'Horodenka': [48.6675, 25.5003], 'Gorodenka': [48.6675, 25.5003], 'Horodence': [48.6675, 25.5003], 'Рівне': [50.6197, 26.2514], 'Rivne': [50.6197, 26.2514], 'Rovno': [50.6197, 26.2514], 'Ровно': [50.6197, 26.2514], 'Równe': [50.6197, 26.2514], "Міжгір'я": [48.5286, 23.5019], 'Mizhhiria': [48.5286, 23.5019], 'Mizhigiria': [48.5286, 23.5019], 'Mizhigirya': [48.5286, 23.5019], 'Межгорье': [48.5286, 23.5019], 'Ökörmező': [48.5286, 23.5019], 'Boureni': [48.5286, 23.5019], 'Mižhiria': [48.5286, 23.5019], 'Volove Pole': [48.5286, 23.5019], 'Богодухів': [50.1608, 35.5164], 'Bohodukhiv': [50.1608, 35.5164], 'Bohoduhov': [50.1608, 35.5164], 'Bogoduhov': [50.1608, 35.5164], 'Богодухов': [50.1608, 35.5164], 'Гола Пристань': [46.5167, 32.5167], 'Hola Prystan': [46.5167, 32.5167], 'Gola Prystan': [46.5167, 32.5167], 'Golaya Pristan': [46.5167, 32.5167], 'Голая Пристань': [46.5167, 32.5167], 'Рожище': [50.9131, 25.27], 'Rozhyshche': [50.9131, 25.27], 'Rozhishche': [50.9131, 25.27], 'Rożyszcze': [50.9131, 25.27], 'Турійськ': [51.0833, 24.5333], 'Turiisk': [51.0833, 24.5333], 'Turiysk': [51.0833, 24.5333], 'Турийск': [51.0833, 24.5333], 'Ємільчине': [50.8708, 27.8028], 'Yemilchyne': [50.8708, 27.8028], 'Yemilchino': [50.8708, 27.8028], 'Емильчино': [50.8708, 27.8028], 'Полтава': [49.5744, 34.5686], 'Poltava': [49.5744, 34.5686], 'Балаклія': [49.4564, 36.8389], 'Balakliia': [49.4564, 36.8389], 'Balakleya': [49.4564, 36.8389], 'Бaлaклeя': [49.4564, 36.8389], 'Ланчин': [48.5475, 24.7517], 'Lanchyn': [48.5475, 24.7517], 'Lanchin': [48.5475, 24.7517], 'Łanczyn': [48.5475, 24.7517], 'Lanciîn': [48.5475, 24.7517], 'Гірник': [48.05, 37.3667], 'Hirnyk': [48.05, 37.3667], 'Gorniak': [48.05, 37.3667], 'Gornyak': [48.05, 37.3667], 'Горняк': [48.05, 37.3667], 'Нові Санжари': [49.3354, 34.3162], 'Novi Sanzhary': [49.3354, 34.3162], 'Novyie Sanzhary': [49.3354, 34.3162], 'Novie Sanzhary': [49.3354, 34.3162], 'Новые Санжары': [49.3354, 34.3162], 'Херсон': [46.6333, 32.6], 'Kherson': [46.6333, 32.6], 'Herson': [46.6333, 32.6], 'Макарів': [50.4667, 29.8167], 'Makariv': [50.4667, 29.8167], 'Makarov': [50.4667, 29.8167], 'Макаров': [50.4667, 29.8167], 'Літин': [49.3255, 28.0872], 'Lityn': [49.3255, 28.0872], 'Litin': [49.3255, 28.0872], 'Литин': [49.3255, 28.0872], 'Ржищів': [49.9611, 31.0436], 'Rzhyshchiv': [49.9611, 31.0436], 'Rzhyshchev': [49.9611, 31.0436], 'Rzhishchev': [49.9611, 31.0436], 'Ржищев': [49.9611, 31.0436], 'Rzyszczów': [49.9611, 31.0436], 'Турбів': [49.3463, 28.7244], 'Turbiv': [49.3463, 28.7244], 'Turbov': [49.3463, 28.7244], 'Турбов': [49.3463, 28.7244], "П'ятихатки": [48.4126, 33.7034], 'Piatykhatky': [48.4126, 33.7034], 'Pyatykhatky': [48.4126, 33.7034], 'Pyatihatki': [48.4126, 33.7034], 'Пятихатки': [48.4126, 33.7034], 'Очаків': [46.6186, 31.5392], 'Ochakiv': [46.6186, 31.5392], 'Ochakov': [46.6186, 31.5392], 'Очаков': [46.6186, 31.5392], 'Özü': [46.6186, 31.5392], 'Oceacov': [46.6186, 31.5392], 'Vozia': [46.6186, 31.5392], 'Alektor': [46.6186, 31.5392], 'Ἀλέκτορος ': [46.6186, 31.5392], 'Білозерка': [46.6333, 32.4333], 'Bilozerka': [46.6333, 32.4333], 'Belozerka': [46.6333, 32.4333], 'Белозерка': [46.6333, 32.4333], 'Чорнобиль': [51.275, 30.221], 'Chornobyl': [51.275, 30.221], 'Chernobyl': [51.275, 30.221], 'Chernobil': [51.275, 30.221], 'Чернобыль': [51.275, 30.221], 'Чернобль': [51.275, 30.221], 'Czarnobyl': [51.275, 30.221], 'Квасилів': [50.5569, 26.2675], 'Kvasyliv': [50.5569, 26.2675], 'Kvasilov': [50.5569, 26.2675], 'Kvasylov': [50.5569, 26.2675], 'Квасилов': [50.5569, 26.2675], 'Kwasiłów': [50.5569, 26.2675], 'Звенигородка': [49.0833, 30.9667], 'Zvenyhorodka': [49.0833, 30.9667], 'Zvenygorodka': [49.0833, 30.9667], 'Zvenigorodka': [49.0833, 30.9667], 'Zwinogródka': [49.0833, 30.9667], 'Сміла': [49.2167, 31.8667], 'Smila': [49.2167, 31.8667], 'Smela': [49.2167, 31.8667], 'Смела': [49.2167, 31.8667], 'Кирнасівка': [48.5914, 28.98], 'Kyrnasivka': [48.5914, 28.98], 'Kyrnasovka': [48.5914, 28.98], 'Kirnasovka': [48.5914, 28.98], 'Кирнасовка': [48.5914, 28.98], 'Ківерці': [50.8331, 25.4614], 'Kivertsi': [50.8331, 25.4614], 'Kivertsy': [50.8331, 25.4614], 'Киверцы': [50.8331, 25.4614], 'Kiwerce': [50.8331, 25.4614], 'Радомишль': [50.4947, 29.2333], 'Radomyshl': [50.4947, 29.2333], 'Radomishl': [50.4947, 29.2333], 'Радомышль': [50.4947, 29.2333], 'Radomyśl': [50.4947, 29.2333], 'Корсунь-Шевче́нківський': [49.4261, 31.2806], 'Korsun-Shevchenkivskyi': [49.4261, 31.2806], 'Korsun-Shevchenkovsky': [49.4261, 31.2806], 'Korsun-Shevchenkovskiy': [49.4261, 31.2806], 'Корсунь-Шевченковский': [49.4261, 31.2806], 'Korsuń Szewczenkowski': [49.4261, 31.2806], 'Великий Бичків': [47.9714, 24.0047], 'Velykyi Bychkiv': [47.9714, 24.0047], 'Velikiy Bychkov': [47.9714, 24.0047], 'Великий Бычков': [47.9714, 24.0047], 'Nagybocskó': [47.9714, 24.0047], 'Nagybocska': [47.9714, 24.0047], 'Bocicoiu Mare': [47.9714, 24.0047], 'Velký Bočkov': [47.9714, 24.0047], 'Veľký Bočkov': [47.9714, 24.0047], 'Bočková': [47.9714, 24.0047], 'Корюківка': [51.7833, 32.25], 'Koriukivka': [51.7833, 32.25], 'Koriukovka': [51.7833, 32.25], 'Korukovka': [51.7833, 32.25], 'Корюковка': [51.7833, 32.25], 'Миколаїв': [46.9667, 32], 'Mykolaiv': [46.9667, 32], 'Mykolayiv': [46.9667, 32], 'Mikolaiv': [46.9667, 32], 'Nikolaev': [46.9667, 32], 'Nikolayev': [46.9667, 32], 'Nikolaiv': [46.9667, 32], 'Николаев': [46.9667, 32], 'Глухів': [51.6765, 33.9078], 'Hlukhiv': [51.6765, 33.9078], 'Glukhov': [51.6765, 33.9078], 'Gluhov': [51.6765, 33.9078], 'Глухов': [51.6765, 33.9078], 'Głuchów': [51.6765, 33.9078], 'Теплик': [48.6594, 29.7539], 'Teplyk': [48.6594, 29.7539], 'Teplik': [48.6594, 29.7539], 'Широке': [47.6882, 33.2654], 'Shyroke': [47.6882, 33.2654], 'Shirokoe': [47.6882, 33.2654], 'Широкое': [47.6882, 33.2654], 'Тернопіль': [49.5667, 25.6], 'Ternopil': [49.5667, 25.6], 'Ternopol': [49.5667, 25.6], 'Тернополь': [49.5667, 25.6], 'Крижопіль': [48.3842, 28.8625], 'Kryzhopil': [48.3842, 28.8625], 'Kryzhopol': [48.3842, 28.8625], 'Крыжополь': [48.3842, 28.8625], 'Брошнів-Осада': [48.9994, 24.1814], 'Broshniv-Osada': [48.9994, 24.1814], 'Broshnev-Osada': [48.9994, 24.1814], 'Брошнев-Осада': [48.9994, 24.1814], 'Тальне': [48.8863, 30.7027], 'Talne': [48.8863, 30.7027], 'Talnoye': [48.8863, 30.7027], 'Тальное': [48.8863, 30.7027], 'Сімферополь': [44.9484, 34.1], 'Simferopol': [44.9484, 34.1], 'Симферополь': [44.9484, 34.1], 'Aqmescit': [44.9484, 34.1], 'Славутич': [51.5206, 30.7569], 'Slavutych': [51.5206, 30.7569], 'Slavutich': [51.5206, 30.7569], 'Луцьк': [50.7478, 25.3244], 'Lutsk': [50.7478, 25.3244], 'Luck': [50.7478, 25.3244], 'Луцк': [50.7478, 25.3244], 'Чуднів': [50.0528, 28.0969], 'Chudniv': [50.0528, 28.0969], 'Chudnov': [50.0528, 28.0969], 'Чуднов': [50.0528, 28.0969], 'Cudnów': [50.0528, 28.0969], 'Пісківка': [50.7078, 29.595], 'Piskivka': [50.7078, 29.595], 'Peskovka': [50.7078, 29.595], 'Песковка': [50.7078, 29.595], 'Муровані Курилівці': [48.7222, 27.515], 'Murovani Kurylivtsi': [48.7222, 27.515], 'Murovanye Kurylovtsy': [48.7222, 27.515], 'Мурованые Куриловцы': [48.7222, 27.515], 'Murowane Kuryłowce': [48.7222, 27.515], 'Джанкой': [45.7086, 34.3933], 'Dzhankoi': [45.7086, 34.3933], 'Jankoy': [45.7086, 34.3933], 'Dshankoj': [45.7086, 34.3933], 'Canköy': [45.7086, 34.3933], 'Павлоград': [48.52, 35.87], 'Pavlohrad': [48.52, 35.87], 'Pavlograd': [48.52, 35.87], 'Благовіщенське': [48.3197, 30.2353], 'Blahovishchenske': [48.3197, 30.2353], 'Blagovishchenske': [48.3197, 30.2353], 'Blahoveshchenskoe': [48.3197, 30.2353], 'Blagoveshchenskoe': [48.3197, 30.2353], 'Благовищенское': [48.3197, 30.2353], 'Летичів': [49.3833, 27.6167], 'Letychiv': [49.3833, 27.6167], 'Letichev': [49.3833, 27.6167], 'Letychev': [49.3833, 27.6167], 'Летичев': [49.3833, 27.6167], 'Latyczów': [49.3833, 27.6167], 'Ічня': [50.85, 32.4], 'Ichnia': [50.85, 32.4], 'Ichnya': [50.85, 32.4], 'Ичня': [50.85, 32.4], 'Мала Виска': [48.65, 31.6333], 'Mala Vyska': [48.65, 31.6333], 'Malaya Vyska': [48.65, 31.6333], 'Malaya Viska': [48.65, 31.6333], 'Малая Виска': [48.65, 31.6333], 'Білопілля': [51.1532, 34.3025], 'Bilopillya': [51.1532, 34.3025], 'Belopolye': [51.1532, 34.3025], 'Белополье': [51.1532, 34.3025], 'Васильківка': [48.2084, 36.0253], 'Vasylkivka': [48.2084, 36.0253], 'Vasilkovka': [48.2084, 36.0253], 'Васильковка': [48.2084, 36.0253], 'Комишуваха': [47.7155, 35.5241], 'Komyshuvakha': [47.7155, 35.5241], 'Kamishevakha': [47.7155, 35.5241], 'Kamyshevakha': [47.7155, 35.5241], 'Камышеваха': [47.7155, 35.5241], 'Черкаси': [49.4444, 32.0597], 'Cherkasy': [49.4444, 32.0597], 'Cherkasi': [49.4444, 32.0597], 'Черкассы': [49.4444, 32.0597], 'Tscherkasy': [49.4444, 32.0597], 'Querkasia': [49.4444, 32.0597], 'Солотвино': [47.9597, 23.8669], 'Solotvyno': [47.9597, 23.8669], 'Solotvino': [47.9597, 23.8669], 'Aknaszlatina': [47.9597, 23.8669], 'Slatina': [47.9597, 23.8669], 'Slatinské Doly': [47.9597, 23.8669], 'Гребінки': [49.9564, 30.1714], 'Hrebinky': [49.9564, 30.1714], 'Grebenki': [49.9564, 30.1714], 'Гребёнки': [49.9564, 30.1714], 'Ратне': [51.65, 24.5333], 'Ratne': [51.65, 24.5333], 'Ratno': [51.65, 24.5333], 'Ратно': [51.65, 24.5333], 'Новгород-Сіверський': [51.9833, 33.2667], 'Novhorod-Siverskyi': [51.9833, 33.2667], 'Novgorod-Siversky': [51.9833, 33.2667], 'Novgorod-Siverskiy': [51.9833, 33.2667], 'Новгород-Северский': [51.9833, 33.2667], 'Миргород': [49.964, 33.6124], 'Myrhorod': [49.964, 33.6124], 'Myrgorod': [49.964, 33.6124], 'Mirhorod': [49.964, 33.6124], 'Mirgorod': [49.964, 33.6124], 'Іваничі': [50.6419, 24.3606], 'Ivanychi': [50.6419, 24.3606], 'Ivanichi': [50.6419, 24.3606], 'Иваничи': [50.6419, 24.3606], 'Iwanicze': [50.6419, 24.3606], 'Романів': [50.1517, 27.9392], 'Romaniv': [50.1517, 27.9392], 'Romanov': [50.1517, 27.9392], 'Romanof': [50.1517, 27.9392], 'Романов': [50.1517, 27.9392], 'Romanów': [50.1517, 27.9392], 'Дзержи́нск': [50.1517, 27.9392], 'Dzierżyńsk': [50.1517, 27.9392], 'Христинівка': [48.8333, 29.9667], 'Khrystynivka': [48.8333, 29.9667], 'Khristinovka': [48.8333, 29.9667], 'Христиновка': [48.8333, 29.9667], 'Червонохригорівка': [47.6233, 34.5309], 'Chervonohryhorivka': [47.6233, 34.5309], 'Chervonogrigorovka': [47.6233, 34.5309], 'Червоногригоровка': [47.6233, 34.5309], 'Токмак': [47.2514, 35.7058], 'Tokmak': [47.2514, 35.7058], 'Маріуполь': [47.1306, 37.5639], 'Mariupol': [47.1306, 37.5639], 'Жданов': [47.1306, 37.5639], 'Zhdanov': [47.1306, 37.5639], 'Мариуполь': [47.1306, 37.5639], 'Μαριούπολη': [47.1306, 37.5639], 'Антрацит': [48.1167, 39.0833], 'Antratsyt': [48.1167, 39.0833], 'Antrasit': [48.1167, 39.0833], 'Antracit': [48.1167, 39.0833], 'Anthratsyt': [48.1167, 39.0833], 'Антрацыт': [48.1167, 39.0833], 'Горохів': [50.4994, 24.765], 'Horokhiv': [50.4994, 24.765], 'Gorokhov': [50.4994, 24.765], 'Gorohov': [50.4994, 24.765], 'Горохов': [50.4994, 24.765], 'Horochów': [50.4994, 24.765], 'Балабине': [47.735, 35.2142], 'Balabyne': [47.735, 35.2142], 'Balabino': [47.735, 35.2142], 'Балабино': [47.735, 35.2142], 'Чорноморськ': [46.3017, 30.6569], 'Chornomorsk': [46.3017, 30.6569], 'Chernomorsk': [46.3017, 30.6569], 'Черноморск': [46.3017, 30.6569], 'Іллічівськ': [46.3017, 30.6569], 'Ильичёвск': [46.3017, 30.6569], 'Illichivsk': [46.3017, 30.6569], 'Судова Вишня': [49.7892, 23.3722], 'Sudova Vyshnia': [49.7892, 23.3722], 'Sudova Vyshnya': [49.7892, 23.3722], 'Sudovaya Vishnya': [49.7892, 23.3722], 'Судовая Вишня': [49.7892, 23.3722], 'Sądowa Wisznia': [49.7892, 23.3722], 'Охтирка': [50.3074, 34.9016], 'Okhtyrka': [50.3074, 34.9016], 'Akhtyrka': [50.3074, 34.9016], 'Ахтырка': [50.3074, 34.9016], 'Харків': [50, 36.2292], 'Kharkiv': [50, 36.2292], 'Kharkov': [50, 36.2292], 'Harkiv': [50, 36.2292], 'Harkov': [50, 36.2292], 'Харьков': [50, 36.2292], 'Тернівка': [48.5231, 36.0826], 'Ternivka': [48.5231, 36.0826], 'Ternovka': [48.5231, 36.0826], 'Терновка': [48.5231, 36.0826], 'Бобровиця': [50.7333, 31.3833], 'Bobrovytsia': [50.7333, 31.3833], 'Bobrovitsia': [50.7333, 31.3833], 'Bobrovytsa': [50.7333, 31.3833], 'Бобровица': [50.7333, 31.3833], 'Стара Вижівка': [51.4396, 24.4373], 'Stara Vyzhivka': [51.4396, 24.4373], 'Staraya Vyzhevka': [51.4396, 24.4373], 'Старая Выжевка': [51.4396, 24.4373], 'Wyżwa Stara': [51.4396, 24.4373], 'Остер': [50.9486, 30.8811], 'Oster': [50.9486, 30.8811], 'Ostior': [50.9486, 30.8811], 'Остёр': [50.9486, 30.8811], 'Носівка': [50.93, 31.58], 'Nosivka': [50.93, 31.58], 'Nosovka': [50.93, 31.58], 'Носовка': [50.93, 31.58], 'Ірпінь': [50.5167, 30.25], 'Irpin': [50.5167, 30.25], 'Irpen': [50.5167, 30.25], 'Ирпень': [50.5167, 30.25], 'Тарутине': [46.1846, 29.1495], 'Tarutyne': [46.1846, 29.1495], 'Tarutino': [46.1846, 29.1495], 'Тарутино': [46.1846, 29.1495], 'Thyratyna': [46.1846, 29.1495], 'Ясинувата': [48.1167, 37.8333], 'Yasynuvata': [48.1167, 37.8333], 'Yasinovataya': [48.1167, 37.8333], 'Ясиноватая': [48.1167, 37.8333], 'Кадіївка': [48.5472, 38.6361], 'Kadiivka': [48.5472, 38.6361], 'Kadiyevka': [48.5472, 38.6361], 'Кадиевка': [48.5472, 38.6361], 'Стаханов': [48.5472, 38.6361], 'Stakhanov': [48.5472, 38.6361], 'Бориспіль': [50.35, 30.95], 'Boryspil': [50.35, 30.95], 'Borispil': [50.35, 30.95], 'Borispol': [50.35, 30.95], 'Борисполь': [50.35, 30.95], 'Bortespil': [50.35, 30.95], 'Боярка': [50.3292, 30.2886], 'Boiarka': [50.3292, 30.2886], 'Boyarka': [50.3292, 30.2886], 'Жашків': [49.25, 30.1], 'Zhashkiv': [49.25, 30.1], 'Zhashkov': [49.25, 30.1], 'Жашков': [49.25, 30.1], 'Берислав': [46.8333, 33.4167], 'Beryslav': [46.8333, 33.4167], 'Berislav': [46.8333, 33.4167], 'Berynslav': [46.8333, 33.4167], 'Озерне': [50.1794, 28.7361], 'Ozerne': [50.1794, 28.7361], 'Ozernoe': [50.1794, 28.7361], 'Озёрное': [50.1794, 28.7361], 'Старокостянтинів': [49.7539, 27.1997], 'Starokostiantyniv': [49.7539, 27.1997], 'Starokonstyantyniv': [49.7539, 27.1997], 'Starokonstantinov': [49.7539, 27.1997], 'Starokostyantyniv': [49.7539, 27.1997], 'Starokostantinov': [49.7539, 27.1997], 'Рава-Руська': [50.25, 23.6167], 'Rava-Ruska': [50.25, 23.6167], 'Rava-Russka': [50.25, 23.6167], 'Rava-Russkaya': [50.25, 23.6167], 'Рава-Русская': [50.25, 23.6167], 'Rawa Ruska': [50.25, 23.6167], 'Погребище': [49.4869, 29.2733], 'Pohrebyshche': [49.4869, 29.2733], 'Pogrebysche': [49.4869, 29.2733], 'Pogrebische': [49.4869, 29.2733], 'Pohrebische': [49.4869, 29.2733], 'Погребыще': [49.4869, 29.2733], 'Рокитня': [49.4869, 29.2733], 'Хорошів': [50.6, 28.45], 'Khoroshiv': [50.6, 28.45], 'Khoroshev': [50.6, 28.45], 'Khorochev': [50.6, 28.45], 'Хорошев': [50.6, 28.45], 'Володарськ-Волинський': [50.6, 28.45], 'Володарск-Волынский': [50.6, 28.45], 'Volodarsk-Volynskyi': [50.6, 28.45], 'Севастополь': [44.6, 33.5333], 'Sevastopol': [44.6, 33.5333], 'Sewastopol': [44.6, 33.5333], 'Sebastopol': [44.6, 33.5333], 'Апостолове': [47.6595, 33.717], 'Apostolove': [47.6595, 33.717], 'Apostolovo': [47.6595, 33.717], 'Апостолово': [47.6595, 33.717], 'Apostolen': [47.6595, 33.717], 'Apostolave': [47.6595, 33.717], 'Ходорів': [49.41, 24.3094], 'Khodoriv': [49.41, 24.3094], 'Khodorov': [49.41, 24.3094], 'Ходоров': [49.41, 24.3094], 'Chodorów': [49.41, 24.3094], 'Тячів': [48.0114, 23.5722], 'Tiachiv': [48.0114, 23.5722], 'Tyachiv': [48.0114, 23.5722], 'Tiachev': [48.0114, 23.5722], 'Tyachev': [48.0114, 23.5722], 'Тячев': [48.0114, 23.5722], 'Зіньків': [50.2081, 34.3668], 'Zinkiv': [50.2081, 34.3668], 'Zenkov': [50.2081, 34.3668], 'Зеньков': [50.2081, 34.3668], 'Приазовське': [46.7378, 35.6416], 'Pryazovske': [46.7378, 35.6416], 'Priazovske': [46.7378, 35.6416], 'Priazovskoe': [46.7378, 35.6416], 'Приазовское': [46.7378, 35.6416], 'Гостомель': [50.5692, 30.2653], 'Hostomel': [50.5692, 30.2653], 'Gostomel': [50.5692, 30.2653], 'Hostemel': [50.5692, 30.2653], 'Михайлівка': [47.2717, 35.2248], 'Mykhailivka': [47.2717, 35.2248], 'Mikhailivka': [47.2717, 35.2248], 'Mikhailovka': [47.2717, 35.2248], 'Mikhaylovka': [47.2717, 35.2248], 'Михайловка': [47.2717, 35.2248], 'Конотоп': [51.2369, 33.2027], 'Konotop': [51.2369, 33.2027], 'Люботин': [49.9489, 35.9306], 'Liubotyn': [49.9489, 35.9306], 'Lyubotyn': [49.9489, 35.9306], 'Надвірна': [48.6333, 24.5833], 'Nadvirna': [48.6333, 24.5833], 'Nadvorna': [48.6333, 24.5833], 'Nadvornaya': [48.6333, 24.5833], 'Nadwirna': [48.6333, 24.5833], 'Надворная': [48.6333, 24.5833], 'Nadworna': [48.6333, 24.5833], 'Миронівка': [49.65, 30.9833], 'Myronivka': [49.65, 30.9833], 'Myronovka': [49.65, 30.9833], 'Mironovka': [49.65, 30.9833], 'Мироновка': [49.65, 30.9833], 'Козятин': [49.7167, 28.8333], 'Koziatyn': [49.7167, 28.8333], 'Kozyatyn': [49.7167, 28.8333], 'Kazatin': [49.7167, 28.8333], 'Казатин': [49.7167, 28.8333], 'Монастирище': [48.99, 29.8011], 'Monastyryshche': [48.99, 29.8011], 'Monastyrishche': [48.99, 29.8011], 'Монастырище': [48.99, 29.8011], 'Жидачів': [49.3833, 24.1333], 'Zhydachiv': [49.3833, 24.1333], 'Zhidachov': [49.3833, 24.1333], 'Жидачов': [49.3833, 24.1333], 'Горішні Плавні': [49.0107, 33.6562], 'Horishni Plavni': [49.0107, 33.6562], 'Gorishni Plavni': [49.0107, 33.6562], 'Gorishnie Plavni': [49.0107, 33.6562], 'Горишние Плавни': [49.0107, 33.6562], 'Комсомольськ': [49.0107, 33.6562], 'Комсомольск': [49.0107, 33.6562], 'Komsomolsk': [49.0107, 33.6562], 'Доброслав': [46.82, 30.945], 'Dobroslav': [46.82, 30.945], 'Kominternivske': [46.82, 30.945], 'Олександрівськ': [48.5833, 39.1833], 'Oleksandrivsk': [48.5833, 39.1833], 'Aleksandrovsk': [48.5833, 39.1833], 'Александровск': [48.5833, 39.1833], 'Березівка': [47.2039, 30.9125], 'Berezivka': [47.2039, 30.9125], 'Berezovka': [47.2039, 30.9125], 'Березовка': [47.2039, 30.9125], 'Нижньогірський': [45.4464, 34.7344], 'Nyzhnohirskyi': [45.4464, 34.7344], 'Nyznohirskyi': [45.4464, 34.7344], 'Nyzhnogirskiy': [45.4464, 34.7344], 'Nizhnegorskiy': [45.4464, 34.7344], 'Нижнегорский': [45.4464, 34.7344], 'Seyitler': [45.4464, 34.7344], 'Карнаухівка': [48.4698, 34.7376], 'Karnaukhivka': [48.4698, 34.7376], 'Karnukhovka': [48.4698, 34.7376], 'Карнауховка': [48.4698, 34.7376], 'Делятин': [48.5286, 24.6236], 'Deliatyn': [48.5286, 24.6236], 'Delyatin': [48.5286, 24.6236], 'Diliatyn': [48.5286, 24.6236], 'Ділятин': [48.5286, 24.6236], "Куп'янськ": [49.7064, 37.6167], 'Kupiansk': [49.7064, 37.6167], 'Kupyansk': [49.7064, 37.6167], 'Купянск': [49.7064, 37.6167], 'Золоте': [48.6833, 38.5167], 'Zolote': [48.6833, 38.5167], 'Zolotoe': [48.6833, 38.5167], 'Zolotoye': [48.6833, 38.5167], 'Золотое': [48.6833, 38.5167], 'Дубровиця': [51.5667, 26.5667], 'Dubrovytsia': [51.5667, 26.5667], 'Dubrovitsya': [51.5667, 26.5667], 'Dubrovytsya': [51.5667, 26.5667], 'Дубровица': [51.5667, 26.5667], 'Dąbrowica': [51.5667, 26.5667]}

In [95]:
def get_locations(message):
        MINIMUM_SIMILARITY = 91
        coord_list = []
        caps = extract_capitalized_words(message)
        for cap in caps:
            matches = process.extract(cap, coord_dict.keys())
            best_match = max(matches, key=lambda x: x[1] if isinstance(x[1], (int, float)) else float('-inf'))
            if best_match[1] >= MINIMUM_SIMILARITY:
                # append [best_match, coords] as list to coord_list
                coord_list.append(best_match[0])
        return coord_list

In [96]:
df['msg_txt'][0]

'  In Kyiv, fragments of a downed rocket damaged a car in Shevchenko district, the city military administration reported. '

In [97]:
df['locations'] = [get_locations(x) for x in df['msg_txt']]

In [98]:
del df['annotation']

In [99]:
del df['idx']

In [100]:
df.head()

,msg_txt,locations
0,"In Kyiv, fragments of a downed rocket damage...","[Kyiv, Shevchenkove]"
1,"Kyiv region is attacked by drones again, air...",[Kyiv]
2,Explosions are also heard in the capital. Ai...,[]
3,Kyiv region is attacked by drones - Kuleba ...,[Kyiv]
4,The rocket that fell in Shevchenkivskyi dist...,[Korsun-Shevchenkivskyi]


In [101]:
good_sample = pd.DataFrame({'msg_txt':['filler'], 'locations':['filler']})

In [102]:
i = 0

for idx, row in df.iterrows():
    if row["locations"] != []:
        #print(row)
        good_sample.loc[i] = row
        i += 1
        #print(good_sample)

In [103]:
good_sample.head()

,msg_txt,locations
0,"In Kyiv, fragments of a downed rocket damage...","[Kyiv, Shevchenkove]"
1,"Kyiv region is attacked by drones again, air...",[Kyiv]
2,Kyiv region is attacked by drones - Kuleba ...,[Kyiv]
3,The rocket that fell in Shevchenkivskyi dist...,[Korsun-Shevchenkivskyi]
4,Current alarm map. Makariv head (Kyiv re...,"[Makariv, Kyiv]"


In [110]:
len(good_sample)

1778

In [111]:
sample = good_sample.iloc[np.random.randint(0,len(good_sample)+1, size=1000)]

In [113]:
sample.head()

,msg_txt,locations
1573,The Armed Forces are working :blue_heart::ye...,[Lviv]
1455,"Robert is the son of Volodymyr Pokotil, an i...","[Volodymyr, Kyiv, Kyiv, Zaporizhzhia]"
29,Over the last week about 350 wounded occupan...,"[Bilovodsk, Luhansk, Zaporizhzhia]"
1181,"In the Berdiansk sector, Ukrainian troops ad...",[Berdiansk]
1648,Zhytomyr and Kyiv regions continue to operat...,"[Zhytomyr, Kyiv]"


In [116]:
sample[0:len(sample)//4].to_csv('dirty-ner-1.csv', index=False)

In [117]:
sample[len(sample)//4:len(sample)//2].to_csv('dirty-ner-2.csv', index=False)

In [118]:
sample[len(sample)//2:3*len(sample)//4].to_csv('dirty-ner-3.csv', index=False)

In [119]:
sample[3*len(sample)//4:].to_csv('dirty-ner-4.csv', index=False)